# Camada Gold - Arquitetura Medalhao

**Objetivo:** Criar tabelas de negocio na camada Gold a partir das tabelas curadas da camada Silver.

**Operacoes:**
- Agregacao de dados para analises de negocios
- Joins entre tabelas Silver para criar visoes consolidadas
- Validacoes de integridade referencial
- Particionamento para otimizacao de queries
- Aplicacao de regras de negocio especificas

## Imports e Configuracoes Iniciais

Importacao de todas as bibliotecas necessarias e configuracao do ambiente Spark.

In [0]:
from datetime import datetime
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.functions import col, to_date, year, upper, trim, coalesce, lit, when, current_timestamp, explode, sequence, month, quarter, dayofweek, dayofmonth, weekofyear, round as spark_round, avg
from pyspark.sql.types import DecimalType, DateType, StringType, IntegerType
from pyspark.sql.utils import AnalysisException

## Setup do Catalogo

Configuracao do catalogo Medalhao e criacao dos schemas Silver e Gold caso nao existam.

In [0]:
%sql
CREATE CATALOG IF NOT EXISTS medalhao;
USE CATALOG medalhao;

CREATE SCHEMA IF NOT EXISTS silver;
CREATE SCHEMA IF NOT EXISTS gold;

USE SCHEMA gold;

## Funcoes Utilitarias

Funcoes auxiliares para gerenciamento de tabelas Delta, validacao de existencia e gravacao com particionamento.

In [0]:
catalog_name = "medalhao"
silver_db_name = "silver"
gold_db_name = "gold"

def table_exists(full_table_name: str) -> bool:
    try:
        spark.table(full_table_name)
        return True
    except AnalysisException:
        return False
    except Exception:
        return False

def ensure_database(db_name: str):
    spark.sql(f"CREATE DATABASE IF NOT EXISTS {db_name}")

def write_delta_overwrite(df, full_table_name: str, partition_cols: list = None):
    writer = df.write.format("delta").mode("overwrite").option("overwriteSchema", "true")
    if partition_cols:
        writer = writer.partitionBy(*partition_cols)
    writer.saveAsTable(full_table_name)

def validar_tabela_silver(tabela_nome: str, catalog: str = "medalhao", schema: str = "silver"):
    full_name = f"{catalog}.{schema}.{tabela_nome}"
    if not table_exists(full_name):
        raise RuntimeError(f"Tabela Silver nao encontrada: {full_name}")
    
    df = spark.table(full_name)
    count = df.count()
    print(f"Leitura: {full_name}")
    print(f"Registros: {count:,}")
    return df

def log_metricas(table_name: str, metrics: dict):
    print(f"\n{'='*80}")
    print(f"METRICAS DE TRANSFORMACAO: {table_name}")
    print(f"Timestamp: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"{'-'*80}")
    for key, value in metrics.items():
        print(f"  {key}: {value}")
    print(f"{'='*80}\n")

def mapear_dia_semana(dia_num: int) -> str:
    mapa = {1: "Domingo", 2: "Segunda-feira", 3: "Terca-feira", 4: "Quarta-feira", 
            5: "Quinta-feira", 6: "Sexta-feira", 7: "Sabado"}
    return mapa.get(dia_num, "Desconhecido")

def mapear_mes(mes_num: int) -> str:
    mapa = {1: "Janeiro", 2: "Fevereiro", 3: "Marco", 4: "Abril", 5: "Maio", 6: "Junho",
            7: "Julho", 8: "Agosto", 9: "Setembro", 10: "Outubro", 11: "Novembro", 12: "Dezembro"}
    return mapa.get(mes_num, "Desconhecido")

# Projeto 1 - Area de Logistica

**Objetivo:** Criar tabela Gold para analise de vendas por localidade dos consumidores.

**Contexto de Negocio:** Facilitar analises de distribuicao geografica de vendas, identificando regioes com maior volume de pedidos e faturamento.

## Questao 1.1 - ft_vendas_consumidor_local

**Descricao:** Tabela fato consolidando pedidos com localizacao dos consumidores para analise geografica de vendas.

**Origem dos Dados:**
- silver.ft_pedido_total (dados de pedidos e valores)
- silver.ft_consumidores (localizacao: cidade e estado)

**Estrutura da Tabela:**
- id_pedido: Identificador unico do pedido (STRING)
- id_consumidor: Identificador do consumidor (STRING)
- valor_total_pedido_brl: Valor total em BRL (DECIMAL(12,2))
- cidade: Cidade do consumidor normalizada (STRING)
- estado: Estado do consumidor normalizado (STRING)
- data_pedido: Data do pedido (DATE)
- ano: Ano extraido para particionamento (INT)

**Estrategia de Particionamento:** Por estado e ano para otimizacao de queries geograficas e temporais.

### Etapa 3.1.1: Carregamento das Tabelas Silver

Carrega as tabelas ft_pedido_total e ft_consumidores da camada Silver, validando existencia e exibindo metadados e amostras para verificacao inicial dos dados de origem.

In [0]:
print("="*80)
print("CARREGAMENTO DAS TABELAS SILVER")
print("="*80)

df_pedido_total = validar_tabela_silver("ft_pedido_total")
df_consumidores = validar_tabela_silver("ft_consumidores")

print("\nEstrutura: ft_pedido_total")
df_pedido_total.printSchema()

print("\nEstrutura: ft_consumidores")
df_consumidores.printSchema()

print("\nAmostra: ft_pedido_total")
display(df_pedido_total.limit(5))

print("\nAmostra: ft_consumidores")
display(df_consumidores.limit(5))

### Etapa 2: Validacoes de Qualidade dos Dados

Executa validacoes criticas de integridade dos dados Silver antes da transformacao Gold. Esta etapa garante que os dados de origem atendem aos requisitos minimos de qualidade, identificando problemas estruturais que podem impactar a camada Gold. As validacoes incluem verificacao de schema, integridade de chaves primarias e estrangeiras, e analise de valores monetarios anomalos.

#### Validacao 2: Integridade das Chaves Primarias

Verifica a presenca de valores nulos nas chaves primarias (id_pedido e id_consumidor). Chaves nulas comprometem a integridade referencial e podem causar perdas de dados nos JOINs da camada Gold. Esta validacao identifica problemas que requerem tratamento especifico durante a transformacao.

In [0]:
print("\nValidacao 2: Integridade das chaves")

null_id_pedido = df_pedido_total.filter(col("id_pedido").isNull()).count()
null_id_consumidor_pedido = df_pedido_total.filter(col("id_consumidor").isNull()).count()
null_id_consumidor_cons = df_consumidores.filter(col("id_consumidor").isNull()).count()

print(f"Nulos: id_pedido={null_id_pedido}, id_consumidor_pedido={null_id_consumidor_pedido}, id_consumidor_cons={null_id_consumidor_cons}")

if null_id_pedido > 0 or null_id_consumidor_pedido > 0 or null_id_consumidor_cons > 0:
    print("Atencao: chaves nulas detectadas, serao tratadas no JOIN")
else:
    print("Verificacao concluida: nenhuma chave nula")

#### Validacao 3: Integridade Referencial (Foreign Keys)

Valida a integridade referencial entre pedidos e consumidores, identificando pedidos orfaos (sem consumidor correspondente). Esta validacao e essencial para garantir a consistencia dos JOINs na camada Gold e identificar problemas de qualidade que podem ter origem em processos upstream ou falhas de integracao.

In [0]:
print("\nValidacao 3: Integridade referencial (FK)")

df_consumidores_ids = df_consumidores.select(col("id_consumidor").alias("fk_consumidor_id")).distinct()

orphan_pedidos = df_pedido_total.join(
    df_consumidores_ids,
    df_pedido_total.id_consumidor == df_consumidores_ids.fk_consumidor_id,
    how="left_anti"
)
orphan_count = orphan_pedidos.count()

print(f"Pedidos orfaos (sem consumidor valido): {orphan_count}")

if orphan_count > 0:
    print(f"Atencao: {orphan_count} pedidos sem consumidor valido serao marcados na Gold")
    print("\nExemplos de pedidos orfaos:")
    display(orphan_pedidos.select("id_pedido", "id_consumidor").limit(5))
else:
    print("Verificacao concluida: todos os pedidos possuem consumidor valido")

#### Validacao 4: Valores Monetarios Anomalos

Analisa a qualidade dos valores monetarios, identificando registros com valores negativos ou nulos. Valores anomalos podem indicar erros de processamento upstream ou problemas de integracao. Esta validacao e fundamental para garantir a confiabilidade das analises financeiras na camada Gold, permitindo decisoes informadas sobre tratamento ou manutencao desses registros para auditoria.

In [0]:
print("\nValidacao 4: Valores monetarios")

valores_negativos = df_pedido_total.filter(col("valor_total_pago_brl") < 0).count()
valores_nulos = df_pedido_total.filter(col("valor_total_pago_brl").isNull()).count()

print(f"Valores negativos: {valores_negativos}")
print(f"Valores nulos: {valores_nulos}")

if valores_negativos > 0 or valores_nulos > 0:
    print("Atencao: valores anomalos detectados, serao mantidos para auditoria")
else:
    print("Verificacao concluida: todos os valores monetarios validos")

### Etapa 3: Transformacao e Enriquecimento dos Dados

Prepara os dados de pedidos com conversao de tipos, normaliza dados de consumidores (uppercase, trim), executa LEFT JOIN para manter todos os pedidos, trata valores ausentes com defaults, adiciona coluna de particionamento (ano) e valida distribuicao final.

In [0]:
print("\n" + "="*80)
print("TRANSFORMACAO PARA CAMADA GOLD")
print("="*80)

print("\nEtapa 3.1: Preparacao dos pedidos")

df_pedidos_preparado = df_pedido_total.select(
    col("id_pedido").cast("string").alias("id_pedido"),
    col("id_consumidor").cast("string").alias("id_consumidor"),
    col("valor_total_pago_brl").cast(DecimalType(12,2)).alias("valor_total_pedido_brl"),
    col("data").alias("data_pedido")
)

total_pedidos_prep = df_pedidos_preparado.count()
print(f"Pedidos preparados: {total_pedidos_prep:,}")

print("\nEtapa 3.2: Preparacao dos consumidores")

df_consumidores_preparado = df_consumidores.select(
    col("id_consumidor").cast("string").alias("id_consumidor"),
    upper(trim(col("cidade"))).alias("cidade"),
    upper(trim(col("estado"))).alias("estado")
).dropDuplicates(["id_consumidor"])

total_consumidores_unicos = df_consumidores_preparado.count()
print(f"Consumidores unicos: {total_consumidores_unicos:,}")

print("\nEtapa 3.1.3: JOIN entre pedidos e consumidores")

df_joined = df_pedidos_preparado.join(
    df_consumidores_preparado,
    on="id_consumidor",
    how="left"
)

total_apos_join = df_joined.count()
print(f"Registros apos JOIN: {total_apos_join:,}")

print("\nEtapa 3.1.4: Tratamento de valores ausentes")

df_gold = df_joined.select(
    col("id_pedido"),
    col("id_consumidor"),
    col("valor_total_pedido_brl"),
    coalesce(col("cidade"), lit("NAO_INFORMADO")).alias("cidade"),
    coalesce(col("estado"), lit("XX")).alias("estado"),
    col("data_pedido")
)

registros_sem_localizacao = df_joined.filter(col("cidade").isNull() | col("estado").isNull()).count()
print(f"Registros com localizacao ausente tratados: {registros_sem_localizacao}")

print("\nEtapa 3.1.5: Adicao de coluna de particionamento")

df_gold = df_gold.withColumn("ano", year(col("data_pedido")))

total_final = df_gold.count()
print(f"Registros finais: {total_final:,}")

print("\nEtapa 3.1.6: Validacao da distribuicao")

print("\nDistribuicao por Estado (Top 10):")
display(df_gold.groupBy("estado").count().orderBy(col("count").desc()).limit(10))

print("\nDistribuicao por Ano:")
display(df_gold.groupBy("ano").count().orderBy("ano"))

### Etapa 3.1.7: Gravacao da Tabela Gold

Grava a tabela ft_vendas_consumidor_local no formato Delta com particionamento por estado e ano, seguida de validacao pos-gravacao incluindo verificacao de schema, tipos de dados, amostra de registros e estatisticas agregadas por particao.

In [0]:
print("\n" + "="*80)
print("GRAVACAO DA TABELA GOLD")
print("="*80)

ensure_database(gold_db_name)

target_table = f"{catalog_name}.{gold_db_name}.ft_vendas_consumidor_local"
partition_cols = ["estado", "ano"]

print(f"\nTabela destino: {target_table}")
print(f"Particionamento: {', '.join(partition_cols)}")

total_a_gravar = df_gold.count()
print(f"Registros a gravar: {total_a_gravar:,}")

print("\nGravando tabela Delta...")
write_delta_overwrite(df_gold, target_table, partition_cols=partition_cols)

print("Gravacao concluida")

print("\n" + "="*80)
print("VALIDACAO POS-GRAVACAO")
print("="*80)

df_validacao = spark.table(target_table)
count_validacao = df_validacao.count()

print(f"\nRegistros gravados: {count_validacao:,}")

if count_validacao != total_a_gravar:
    print(f"ATENCAO: Divergencia de contagem (esperado: {total_a_gravar:,})")

print(f"\nSchema da tabela:")
df_validacao.printSchema()

print("\nTipos de dados:")
for field in df_validacao.schema.fields:
    print(f"  {field.name}: {field.dataType}")

print("\nAmostra dos dados (20 registros mais recentes):")
display(df_validacao.orderBy(col("data_pedido").desc()).limit(20))

print("\nEstatisticas por Estado (Top 10):")
display(
    df_validacao.groupBy("estado")
    .agg(
        F.count("*").alias("total_pedidos"),
        F.sum("valor_total_pedido_brl").alias("valor_total_brl"),
        F.avg("valor_total_pedido_brl").alias("valor_medio_brl")
    )
    .orderBy(col("total_pedidos").desc())
    .limit(10)
)

print("\nEstatisticas por Ano:")
display(
    df_validacao.groupBy("ano")
    .agg(
        F.count("*").alias("total_pedidos"),
        F.sum("valor_total_pedido_brl").alias("valor_total_brl"),
        F.avg("valor_total_pedido_brl").alias("valor_medio_brl")
    )
    .orderBy("ano")
)

metricas_finais = {
    "Tabela": target_table,
    "Registros totais": f"{count_validacao:,}",
    "Particionamento": ", ".join(partition_cols),
    "Origem Silver": "ft_pedido_total, ft_consumidores",
    "Formato": "Delta Lake",
    "Status": "Concluido com sucesso"
}

log_metricas("ft_vendas_consumidor_local", metricas_finais)

## Questao 1.2 - view_total_compras_por_consumidor

**Descricao:** View que consolida o total de compras e o valor total vendido por cidade e estado, permitindo identificar localidades com maior concentracao de vendas.

**Origem dos Dados:**
- gold.ft_vendas_consumidor_local (tabela fato criada na Questao 1.1)

**Estrutura da View:**
- cidade: Cidade do cliente (STRING)
- estado: Estado do cliente (STRING)
- quantidade_vendas: Quantidade de pedidos realizados na localidade (BIGINT)
- valor_total_localidade: Soma total dos valores pagos em BRL (DECIMAL)

**Proposito:** Esta view facilita analises geograficas de vendas, identificando regioes com maior volume de pedidos e faturamento. Conecta-se diretamente a tabela Gold ft_vendas_consumidor_local para fornecer metricas consolidadas por localidade, essenciais para estrategias de expansao e logistica.

In [0]:
print("="*80)
print("CRIACAO DA VIEW: view_total_compras_por_consumidor")
print("="*80)

view_name = f"{catalog_name}.{gold_db_name}.view_total_compras_por_consumidor"
source_table = f"{catalog_name}.{gold_db_name}.ft_vendas_consumidor_local"

print(f"\nView: {view_name}")
print(f"Origem: {source_table}")

df_source = spark.table(source_table)

df_view = df_source.groupBy("cidade", "estado").agg(
    F.count("id_pedido").alias("quantidade_vendas"),
    F.sum("valor_total_pedido_brl").alias("valor_total_localidade")
)

df_view.write.format("delta").mode("overwrite").saveAsTable(view_name)
print("\nView criada com sucesso")

print("\n" + "="*80)
print("VALIDACAO DA VIEW")
print("="*80)

df_view_read = spark.table(view_name)
count_view = df_view_read.count()

print(f"\nTotal de localidades: {count_view:,}")

print("\nSchema da view:")
df_view_read.printSchema()

print("\nTop 20 localidades por quantidade de vendas:")
display(df_view_read.orderBy(col("quantidade_vendas").desc()).limit(20))

print("\nTop 20 localidades por valor total:")
display(df_view_read.orderBy(col("valor_total_localidade").desc()).limit(20))

print("\n" + "="*80)
print("CONSULTA: Total de vendas por estado")
print("="*80)

df_vendas_estado = df_view_read.groupBy("estado").agg(
    F.sum("quantidade_vendas").alias("total_vendas_estado"),
    F.sum("valor_total_localidade").alias("valor_total_estado")
).orderBy(col("total_vendas_estado").desc())

print("\nTotal de vendas por estado:")
display(df_vendas_estado)

metricas_view = {
    "View": view_name,
    "Localidades unicas": f"{count_view:,}",
    "Origem": source_table,
    "Agrupamento": "cidade, estado",
    "Status": "Criada com sucesso"
}

log_metricas("view_total_compras_por_consumidor", metricas_view)

# Projeto 2 - Area de Logistica

**Objetivo:** Analisar atrasos de entregas identificando regioes com maior ocorrencia de atrasos e vendedores associados.

**Contexto de Negocio:** A equipe de Logistica enfrenta aumento nos indices de atraso e precisa identificar gargalos na cadeia de entrega, regioes problematicas e vendedores com maior incidencia de atrasos para otimizar processos logisticos.

## Questao 2.1 - ft_atrasos_pedidos_local_vendedor

**Descricao:** Tabela fato contendo informacoes logisticas de cada pedido, incluindo metricas de atraso e identificacao de vendedor e localizacao.

**Origem dos Dados:**
- silver.ft_pedidos (datas de pedido, compra, entrega estimada e real)
- silver.ft_consumidores (localizacao: cidade e estado)
- silver.ft_itens_pedidos (identificacao do vendedor)

**Estrutura da Tabela:**
- id_pedido: Identificador unico do pedido (STRING)
- id_vendedor: Identificador do vendedor responsavel (STRING)
- id_consumidor: Identificador do cliente (STRING)
- entrega_no_prazo: Classificacao da entrega - "Sim", "Nao" ou "Nao Entregue" (STRING)
- tempo_entrega_dias: Tempo real de entrega em dias (INT)
- tempo_entrega_estimado_dias: Tempo estimado de entrega em dias (INT)
- cidade: Cidade do cliente normalizada (STRING)
- estado: Estado do cliente normalizado (STRING)

**Logica de Composicao:**
- entrega_no_prazo: "Sim" quando data_entrega <= data_entrega_estimada; "Nao" quando data_entrega > data_entrega_estimada; "Nao Entregue" quando data_entrega e nula
- tempo_entrega_dias: datediff(data_entrega, data_pedido_compra) - calculado apenas quando entrega foi realizada
- tempo_entrega_estimado_dias: datediff(data_entrega_estimada, data_pedido_compra)
- id_vendedor: obtido da primeira ocorrencia em ft_itens_pedidos (pedidos podem ter multiplos vendedores, seleciona-se o primeiro)

**Proposito:** Esta tabela permite analise detalhada de atrasos por regiao e vendedor, identificando gargalos logisticos e padroes de desempenho para otimizacao da cadeia de entrega.

### Etapa 3.2.1: Carregamento das Tabelas Silver (Projeto 2)

Carrega as tres tabelas Silver necessarias para analise de atrasos: ft_pedidos (informacoes de datas), ft_consumidores (localizacao) e ft_itens_pedidos (identificacao do vendedor). Este carregamento estabelece a base de dados para calcular metricas logisticas e identificar gargalos na cadeia de entrega.

In [0]:
print("="*80)
print("CRIACAO DA TABELA: ft_atrasos_pedidos_local_vendedor")
print("="*80)

print("\nCarregamento das tabelas Silver")

df_pedidos = validar_tabela_silver("ft_pedidos")
df_consumidores = validar_tabela_silver("ft_consumidores")
df_itens_pedidos = validar_tabela_silver("ft_itens_pedidos")

### Etapa 3.2.2: Preparacao dos Dados de Vendedores

Extrai e deduplica os dados de vendedores da tabela ft_itens_pedidos. Como um pedido pode ter multiplos itens de diferentes vendedores, esta etapa seleciona o primeiro vendedor por pedido para simplificar a analise. Esta abordagem permite identificar o vendedor primario responsavel pelo pedido na analise de atrasos.

In [0]:
print("\nPreparacao dos dados de vendedores")

df_vendedores = df_itens_pedidos.select(
    col("id_pedido"),
    col("id_vendedor")
).dropDuplicates(["id_pedido"])

total_vendedores = df_vendedores.count()
print(f"Pedidos com vendedor identificado: {total_vendedores:,}")

### Etapa 3.2.3: Preparacao dos Dados de Consumidores

Normaliza os dados de localizacao dos consumidores aplicando uppercase e trim para garantir consistencia. Remove duplicatas de id_consumidor para manter apenas um registro por cliente. Esta padronizacao e essencial para analises geograficas consistentes e agregacoes precisas por cidade e estado na identificacao de regioes com maior incidencia de atrasos.

In [0]:
print("\nPreparacao dos dados de consumidores")

df_consumidores_prep = df_consumidores.select(
    col("id_consumidor").cast("string").alias("id_consumidor"),
    upper(trim(col("cidade"))).alias("cidade"),
    upper(trim(col("estado"))).alias("estado")
).dropDuplicates(["id_consumidor"])

total_consumidores = df_consumidores_prep.count()
print(f"Consumidores unicos: {total_consumidores:,}")

### Etapa 3.2.4: Calculo das Metricas Logisticas

Calcula as metricas essenciais para analise de atrasos: tempo_entrega_dias (diferenca entre data_entrega e data_pedido_compra), tempo_entrega_estimado_dias (diferenca entre data_entrega_estimada e data_pedido_compra), e entrega_no_prazo (classificacao "Sim", "Nao" ou "Nao Entregue"). Estas metricas sao fundamentais para identificar padroes de atraso e avaliar performance logistica por regiao e vendedor.

In [0]:
print("\nCalculo das metricas logisticas")

df_pedidos_metricas = df_pedidos.select(
    col("id_pedido").cast("string").alias("id_pedido"),
    col("id_consumidor").cast("string").alias("id_consumidor"),
    col("entrega_no_prazo"),
    when(col("diferenca_entrega_dias").isNotNull(), col("diferenca_entrega_dias"))
        .otherwise(lit(None)).cast("int").alias("tempo_entrega_dias"),
    when(col("tempo_entrega_estimado_dias").isNotNull(), col("tempo_entrega_estimado_dias"))
        .otherwise(lit(None)).cast("int").alias("tempo_entrega_estimado_dias")
)

total_pedidos = df_pedidos_metricas.count()
print(f"Pedidos processados: {total_pedidos:,}")

### Etapa 3.2.5: Consolidacao e Tratamento de Dados Ausentes

Executa JOINs left entre pedidos, vendedores e consumidores para manter todos os pedidos mesmo sem vendedor ou localizacao identificados. Aplica coalesce para tratar valores ausentes com defaults padronizados ("NAO_INFORMADO" para vendedor e cidade, "XX" para estado). Esta abordagem garante completude dos dados para analise sem perda de registros por ausencia de informacoes complementares.

In [0]:
print("\nConsolidacao dos dados")

df_gold_temp = df_pedidos_metricas.join(
    df_vendedores,
    on="id_pedido",
    how="left"
).join(
    df_consumidores_prep,
    on="id_consumidor",
    how="left"
)

df_gold = df_gold_temp.select(
    col("id_pedido"),
    coalesce(col("id_vendedor"), lit("NAO_INFORMADO")).alias("id_vendedor"),
    col("id_consumidor"),
    col("entrega_no_prazo"),
    col("tempo_entrega_dias"),
    col("tempo_entrega_estimado_dias"),
    coalesce(col("cidade"), lit("NAO_INFORMADO")).alias("cidade"),
    coalesce(col("estado"), lit("XX")).alias("estado")
)

total_final = df_gold.count()
print(f"Registros finais: {total_final:,}")

### Etapa 3.2.6: Validacao da Distribuicao dos Dados

Valida a distribuicao dos dados antes da gravacao, analisando a proporcao de entregas no prazo versus atrasadas por status e a distribuicao geografica por estado. Esta validacao preliminar permite identificar inconsistencias ou padroes inesperados antes da persistencia na camada Gold, garantindo qualidade dos dados para analises posteriores.

In [0]:
print("\nValidacao da distribuicao")

print("\nDistribuicao por status de entrega:")
display(df_gold.groupBy("entrega_no_prazo").count().orderBy("entrega_no_prazo"))

print("\nDistribuicao por Estado (Top 10):")
display(df_gold.groupBy("estado").count().orderBy(col("count").desc()).limit(10))

### Etapa 3.2.7: Gravacao da Tabela Gold

Persiste a tabela ft_atrasos_pedidos_local_vendedor no formato Delta Lake no schema Gold. A gravacao em modo overwrite garante atualizacao completa dos dados, mantendo apenas a versao mais recente. Esta tabela Gold servira como fonte para analises de atrasos, identificacao de gargalos logisticos e avaliacao de performance por regiao e vendedor.

In [0]:
print("\n" + "="*80)
print("GRAVACAO DA TABELA GOLD")
print("="*80)

target_table = f"{catalog_name}.{gold_db_name}.ft_atrasos_pedidos_local_vendedor"

print(f"\nTabela destino: {target_table}")

total_a_gravar = df_gold.count()
print(f"Registros a gravar: {total_a_gravar:,}")

print("\nGravando tabela Delta...")
write_delta_overwrite(df_gold, target_table)

print("Gravacao concluida")

### Etapa 3.2.8: Validacao Pos-Gravacao e Analise de Atrasos

Valida a tabela gravada verificando contagem de registros, schema e tipos de dados. Exibe amostra dos dados e calcula estatisticas agregadas de atrasos por status de entrega, incluindo tempo medio de entrega real versus estimado. Esta validacao final confirma a integridade da tabela Gold e fornece insights preliminares sobre a distribuicao de atrasos para direcionar analises subsequentes.

In [0]:
print("\n" + "="*80)
print("VALIDACAO POS-GRAVACAO")
print("="*80)

df_validacao = spark.table(target_table)
count_validacao = df_validacao.count()

print(f"\nRegistros gravados: {count_validacao:,}")

if count_validacao != total_a_gravar:
    print(f"ATENCAO: Divergencia de contagem (esperado: {total_a_gravar:,})")

print(f"\nSchema da tabela:")
df_validacao.printSchema()

print("\nAmostra dos dados (20 registros):")
display(df_validacao.limit(20))

print("\nEstatisticas de atrasos:")
display(
    df_validacao.groupBy("entrega_no_prazo")
    .agg(
        F.count("*").alias("total_pedidos"),
        F.avg("tempo_entrega_dias").alias("tempo_medio_entrega_dias"),
        F.avg("tempo_entrega_estimado_dias").alias("tempo_medio_estimado_dias")
    )
    .orderBy("entrega_no_prazo")
)

metricas_finais = {
    "Tabela": target_table,
    "Registros totais": f"{count_validacao:,}",
    "Origem Silver": "ft_pedidos, ft_consumidores, ft_itens_pedidos",
    "Formato": "Delta Lake",
    "Status": "Concluido com sucesso"
}

log_metricas("ft_atrasos_pedidos_local_vendedor", metricas_finais)

## Questao 2.2 - Views Analiticas de Logistica

**Objetivo:** Criar views Gold para analise de desempenho logistico por localidade e vendedor.

**Contexto de Negocio:** A equipe de Logistica necessita de metricas agregadas para identificar regioes problematicas e vendedores com baixa pontualidade, permitindo acoes corretivas direcionadas e otimizacao da cadeia de entrega.

**Views a serem criadas:**
1. view_tempo_medio_entrega_localidade - Analise de tempos de entrega por regiao
2. view_vendedor_pontualidade - Analise de desempenho e pontualidade por vendedor

### Questao 2.2.1 - view_tempo_medio_entrega_localidade

**Descricao:** View agregando metricas de tempo de entrega por cidade e estado, comparando performance real versus estimada.

**Origem dos Dados:**
- gold.ft_atrasos_pedidos_local_vendedor

**Estrutura da View:**
- cidade: Cidade do cliente normalizada (STRING)
- estado: Estado do cliente normalizado (STRING)
- tempo_medio_entrega: Media do tempo real de entrega em dias, arredondada com 2 casas decimais (DECIMAL)
- tempo_medio_estimado: Media do tempo estimado de entrega em dias, arredondada com 2 casas decimais (DECIMAL)
- entrega_maior_que_estimado: Indicador se tempo real excede tempo estimado - "SIM" ou "NAO" (STRING)

**Proposito:** Esta view permite identificar regioes onde o tempo real de entrega consistentemente excede as estimativas, indicando gargalos logisticos geograficos que requerem atencao. A comparacao entre tempo real e estimado por localidade auxilia no planejamento de melhorias operacionais e ajuste de expectativas de entrega.

#### Etapa 1: Carregamento e Agregacao por Localidade

Carrega a tabela gold.ft_atrasos_pedidos_local_vendedor e calcula as medias de tempo de entrega real e estimado por cidade e estado. Arredonda as medias para 2 casas decimais conforme requisito de padronizacao de metricas. Cria indicador booleano comparando se o tempo medio real excede o tempo medio estimado, classificando regioes com desempenho abaixo do esperado.

In [0]:
print("="*80)
print("[ETAPA 1] Criacao da view: view_tempo_medio_entrega_localidade")
print("="*80)

source_table = f"{catalog_name}.{gold_db_name}.ft_atrasos_pedidos_local_vendedor"
view_name = f"{catalog_name}.{gold_db_name}.view_tempo_medio_entrega_localidade"

print(f"\nOrigem: {source_table}")
print(f"Destino: {view_name}")

df_source = spark.table(source_table)

df_source_filtrado = df_source.filter(
    (col("entrega_no_prazo").isin("Sim", "Nao")) &
    col("tempo_entrega_dias").isNotNull() &
    col("tempo_entrega_estimado_dias").isNotNull()
)

total_pedidos_filtrados = df_source_filtrado.count()
print(f"\nPedidos validos para calculo (excluindo 'Nao Entregue' e NULLs): {total_pedidos_filtrados:,}")

df_view_localidade = df_source_filtrado.groupBy("cidade", "estado").agg(
    spark_round(F.avg("tempo_entrega_dias"), 2).alias("tempo_medio_entrega"),
    spark_round(F.avg("tempo_entrega_estimado_dias"), 2).alias("tempo_medio_estimado")
).withColumn(
    "entrega_maior_que_estimado",
    when(col("tempo_medio_entrega") > col("tempo_medio_estimado"), lit("SIM"))
        .otherwise(lit("NAO"))
)

total_localidades = df_view_localidade.count()
print(f"\nLocalidades processadas: {total_localidades:,}")

df_view_localidade.write.format("delta").mode("overwrite").saveAsTable(view_name)
print(f"\n[ETAPA 1] View criada com sucesso")

#### Etapa 2: Validacao e Analise da View de Localidade

Valida a view criada verificando schema, tipos de dados e distribuicao dos indicadores. Exibe amostra de localidades com tempo real excedendo estimativa para validacao da logica de negocio. Calcula estatisticas agregadas para identificar proporcao de regioes problematicas onde entregas consistentemente ultrapassam o prazo estimado.

In [0]:
print("\n" + "="*80)
print("[ETAPA 2] Validacao da view: view_tempo_medio_entrega_localidade")
print("="*80)

df_view_read = spark.table(view_name)
count_view = df_view_read.count()

print(f"\nTotal de localidades na view: {count_view:,}")

print("\nSchema da view:")
df_view_read.printSchema()

print("\n[ETAPA 2] Amostra de localidades com entrega acima do estimado:")
display(
    df_view_read.filter(col("entrega_maior_que_estimado") == "SIM")
    .orderBy(col("tempo_medio_entrega").desc())
    .limit(20)
)

print("\n[ETAPA 2] Distribuicao por indicador:")
display(
    df_view_read.groupBy("entrega_maior_que_estimado")
    .agg(F.count("*").alias("total_localidades"))
    .orderBy("entrega_maior_que_estimado")
)

print("\n[ETAPA 2] Top 10 estados com maior tempo medio de entrega:")
display(
    df_view_read.groupBy("estado")
    .agg(
        F.round(F.avg("tempo_medio_entrega"), 2).alias("tempo_medio_estado"),
        F.count("*").alias("total_cidades")
    )
    .orderBy(col("tempo_medio_estado").desc())
    .limit(10)
)

metricas_view_localidade = {
    "View": view_name,
    "Total de localidades": f"{count_view:,}",
    "Origem": source_table,
    "Metricas": "tempo_medio_entrega (2 casas), tempo_medio_estimado (2 casas)",
    "Status": "Criada e validada com sucesso"
}

log_metricas("view_tempo_medio_entrega_localidade", metricas_view_localidade)

### VALIDACAO ADICIONAL: Verificacao de Exclusoes

Valida que a view excluiu corretamente pedidos "Nao Entregue" e valores NULL, garantindo que apenas tempos reais de entrega sejam considerados nas medias por localidade.

In [ ]:
print("\n" + "="*80)
print("VALIDACAO: Verificacao de registros excluidos")
print("="*80)

# Contar pedidos "Nao Entregue" na tabela fonte
df_source_original = spark.table(source_table)
total_nao_entregue = df_source_original.filter(col("entrega_no_prazo") == "Nao Entregue").count()
total_null_tempo_entrega = df_source_original.filter(col("tempo_entrega_dias").isNull()).count()
total_null_tempo_estimado = df_source_original.filter(col("tempo_entrega_estimado_dias").isNull()).count()

print(f"\nRegistros excluidos da view:")
print(f"  - Pedidos 'Nao Entregue': {total_nao_entregue:,}")
print(f"  - Pedidos com tempo_entrega_dias NULL: {total_null_tempo_entrega:,}")
print(f"  - Pedidos com tempo_entrega_estimado_dias NULL: {total_null_tempo_estimado:,}")

# Verificar que a view nao tem NULLs nas medias
df_view_check = spark.table(view_name)
medias_null = df_view_check.filter(
    col("tempo_medio_entrega").isNull() | col("tempo_medio_estimado").isNull()
).count()

if medias_null > 0:
    print(f"\n  ALERTA: {medias_null} localidades com medias NULL na view")
else:
    print(f"\n  OK: Nenhuma localidade com medias NULL na view")

print("\nAmostra de localidades com entregas validas:")
display(df_view_check.orderBy(col("tempo_medio_entrega").desc()).limit(5))

### Questao 2.2.2 - view_vendedor_pontualidade

**Descricao:** View agregando metricas de pontualidade por vendedor, calculando taxa de atraso para identificar vendedores com baixo desempenho logistico.

**Origem dos Dados:**
- gold.ft_atrasos_pedidos_local_vendedor

**Estrutura da View:**
- id_vendedor: Identificador do vendedor (STRING)
- total_pedidos: Quantidade total de pedidos do vendedor (BIGINT)
- total_atrasados: Quantidade de pedidos com atraso (entrega_no_prazo = "Nao") (BIGINT)
- percentual_atraso: Proporcao percentual de pedidos atrasados, arredondada com 2 casas decimais (DECIMAL)

**Regras de Negocio:**
- Atraso e definido como entrega_no_prazo = "Nao"
- percentual_atraso = (total_atrasados / total_pedidos) * 100
- Vendedores com percentual_atraso alto requerem investigacao e acoes corretivas

**Proposito:** Esta view permite identificar vendedores com maior incidencia de atrasos, facilitando acoes de melhoria operacional direcionadas. A metrica de percentual de atraso normaliza o desempenho independente do volume de pedidos, permitindo comparacao justa entre vendedores de diferentes portes.

#### Etapa 1: Calculo de Metricas de Pontualidade por Vendedor

Carrega a tabela gold.ft_atrasos_pedidos_local_vendedor e calcula tres metricas essenciais por vendedor: total de pedidos, total de pedidos atrasados (entrega_no_prazo = "Nao") e percentual de atraso. O percentual e calculado como proporcao de pedidos atrasados sobre o total, multiplicado por 100 e arredondado para 2 casas decimais. Esta agregacao permite identificar vendedores com desempenho logistico abaixo do esperado.

In [0]:
print("="*80)
print("[ETAPA 1] Criacao da view: view_vendedor_pontualidade")
print("="*80)

view_name_vendedor = f"{catalog_name}.{gold_db_name}.view_vendedor_pontualidade"

print(f"\nOrigem: {source_table}")
print(f"Destino: {view_name_vendedor}")

df_source_vendedor = spark.table(source_table)

df_view_vendedor = df_source_vendedor.groupBy("id_vendedor").agg(
    F.count("*").alias("total_pedidos"),
    F.sum(when(col("entrega_no_prazo") == "Nao", 1).otherwise(0)).alias("total_atrasados")
).withColumn(
    "percentual_atraso",
    F.round((col("total_atrasados") / col("total_pedidos")) * 100, 2)
)

total_vendedores = df_view_vendedor.count()
print(f"\nVendedores processados: {total_vendedores:,}")

df_view_vendedor.write.format("delta").mode("overwrite").saveAsTable(view_name_vendedor)
print(f"\n[ETAPA 1] View criada com sucesso")

#### Etapa 2: Validacao e Analise da View de Pontualidade

Valida a view criada verificando schema, tipos de dados e distribuicao de percentuais de atraso. Exibe vendedores com maior taxa de atraso para identificacao de casos criticos que requerem atencao imediata. Calcula estatisticas agregadas incluindo media e mediana de percentual de atraso, permitindo benchmark de desempenho e identificacao de outliers na pontualidade de entregas.

In [0]:
print("\n" + "="*80)
print("[ETAPA 2] Validacao da view: view_vendedor_pontualidade")
print("="*80)

df_view_vendedor_read = spark.table(view_name_vendedor)
count_vendedores = df_view_vendedor_read.count()

print(f"\nTotal de vendedores na view: {count_vendedores:,}")

print("\nSchema da view:")
df_view_vendedor_read.printSchema()

print("\n[ETAPA 2] Top 20 vendedores com maior percentual de atraso:")
display(
    df_view_vendedor_read.orderBy(col("percentual_atraso").desc())
    .limit(20)
)

print("\n[ETAPA 2] Top 20 vendedores com melhor pontualidade:")
display(
    df_view_vendedor_read.orderBy(col("percentual_atraso").asc())
    .limit(20)
)

print("\n[ETAPA 2] Estatisticas gerais de pontualidade:")
display(
    df_view_vendedor_read.select(
        F.round(F.avg("percentual_atraso"), 2).alias("percentual_medio_atraso"),
        F.round(F.min("percentual_atraso"), 2).alias("percentual_minimo"),
        F.round(F.max("percentual_atraso"), 2).alias("percentual_maximo"),
        F.sum("total_pedidos").alias("total_geral_pedidos"),
        F.sum("total_atrasados").alias("total_geral_atrasados")
    )
)

print("\n[ETAPA 2] Distribuicao de vendedores por faixa de atraso:")
display(
    df_view_vendedor_read.select(
        when(col("percentual_atraso") < 10, lit("0-10%"))
        .when(col("percentual_atraso") < 20, lit("10-20%"))
        .when(col("percentual_atraso") < 30, lit("20-30%"))
        .when(col("percentual_atraso") < 40, lit("30-40%"))
        .otherwise(lit("40%+")).alias("faixa_atraso")
    ).groupBy("faixa_atraso")
    .agg(F.count("*").alias("total_vendedores"))
    .orderBy("faixa_atraso")
)

metricas_view_vendedor = {
    "View": view_name_vendedor,
    "Total de vendedores": f"{count_vendedores:,}",
    "Origem": source_table,
    "Metricas": "total_pedidos, total_atrasados, percentual_atraso (2 casas)",
    "Status": "Criada e validada com sucesso"
}

log_metricas("view_vendedor_pontualidade", metricas_view_vendedor)

# Projeto 3 - Area Comercial (Analises de Vendas por Periodo)

**Objetivo:** Criar dimensao de tempo para suportar analises temporais de vendas com granularidade diaria.

**Contexto de Negocio:** A equipe Comercial necessita de uma dimensao de tempo estruturada para realizar analises de vendas por diferentes perspectivas temporais (ano, trimestre, mes, semana, dia). Esta dimensao permite agregacoes temporais consistentes e facilita identificacao de sazonalidades e tendencias de vendas ao longo do tempo.

## Questao 3.1 - dm_tempo (Dimensao de Tempo)

**Descricao:** Tabela dimensao contendo todos os dias dentro de um periodo definido, com atributos temporais para analises multidimensionais de vendas.

**Estrutura da Tabela:**
- sk_tempo: Chave substituta - a propria data (DATE)
- ano: Ano extraido da data (INT)
- trimestre: Trimestre do ano, valores 1 a 4 (INT)
- mes: Mes do ano, valores 1 a 12 (INT)
- semana_do_ano: Semana do ano conforme ISO 8601 (INT)
- dia: Dia do mes (INT)
- dia_da_semana_num: Dia da semana numerico, 1=Domingo, 7=Sabado (INT)
- dia_da_semana_nome: Nome do dia em portugues (STRING)
- mes_nome: Nome do mes em portugues (STRING)
- eh_fim_de_semana: Indicador "Sim" para sabado/domingo, "Nao" caso contrario (STRING)

**Estrategia de Geracao:** Utiliza sequence + explode para gerar array de datas entre data_inicio e data_fim, depois explode para criar um registro por dia. Mapeamento de nomes em portugues garante consistencia independente de locale do sistema.

**Particionamento:** Por ano e mes para otimizacao de queries que filtram por periodos especificos.

### Etapa 1: Definicao de Parametros

Define o periodo de abrangencia da dimensao tempo atraves das datas de inicio e fim. O periodo deve cobrir todas as datas presentes nas tabelas de fato para garantir integridade referencial nas analises temporais. Esta etapa estabelece os limites para a geracao da sequencia de datas que comporao a dimensao.

In [0]:
print("="*80)
print("[ETAPA 3.3.1] Definicao de parametros para dimensao tempo")
print("="*80)

data_inicio = '2016-01-01'
data_fim = '2018-12-31'

print(f"\nParametros definidos:")
print(f"  data_inicio: {data_inicio}")
print(f"  data_fim: {data_fim}")

### Etapa 2: Geracao da Sequencia de Datas

Utiliza as funcoes sequence e explode do PySpark para gerar todas as datas entre data_inicio e data_fim. A funcao sequence cria um array de datas com granularidade diaria, e explode transforma cada elemento do array em uma linha individual. Esta abordagem e eficiente e escalavel, gerando a sequencia completa de datas em uma unica operacao sem necessidade de loops ou iteracoes manuais.

In [0]:
print("\n" + "="*80)
print("[ETAPA 3.3.2] Gerando sequencia de datas")
print("="*80)

df_sequencia = spark.sql(f"""
    SELECT explode(sequence(to_date('{data_inicio}'), to_date('{data_fim}'), interval 1 day)) as sk_tempo
""")

total_datas = df_sequencia.count()
print(f"\nDatas geradas: {total_datas:,}")
print(f"Data minima: {data_inicio}")
print(f"Data maxima: {data_fim}")

### Etapa 3: Calculo de Atributos Temporais

Enriquece cada data com atributos temporais essenciais para analises multidimensionais: ano, trimestre, mes, semana do ano, dia do mes e dia da semana. Cria mapeamentos explicitos para nomes de dias e meses em portugues usando UDFs registradas, garantindo consistencia independente de configuracoes de locale do sistema. Adiciona indicador de fim de semana para facilitar analises de sazonalidade semanal. Todos os atributos sao calculados de forma deterministica a partir da data base.

In [0]:
print("\n" + "="*80)
print("[ETAPA 3.3.3] Calculando atributos temporais")
print("="*80)

from pyspark.sql.types import StringType

spark.udf.register("mapear_dia_semana_udf", mapear_dia_semana, StringType())
spark.udf.register("mapear_mes_udf", mapear_mes, StringType())

df_tempo = df_sequencia.withColumn("ano", year(col("sk_tempo"))) \
    .withColumn("trimestre", quarter(col("sk_tempo"))) \
    .withColumn("mes", month(col("sk_tempo"))) \
    .withColumn("semana_do_ano", weekofyear(col("sk_tempo"))) \
    .withColumn("dia", dayofmonth(col("sk_tempo"))) \
    .withColumn("dia_da_semana_num", dayofweek(col("sk_tempo"))) \
    .withColumn("dia_da_semana_nome", F.expr("mapear_dia_semana_udf(dia_da_semana_num)")) \
    .withColumn("mes_nome", F.expr("mapear_mes_udf(mes)")) \
    .withColumn("eh_fim_de_semana", 
                when((col("dia_da_semana_num") == 1) | (col("dia_da_semana_num") == 7), lit("Sim"))
                .otherwise(lit("Nao")))

print("\nAtributos calculados:")
print("  - ano, trimestre, mes, semana_do_ano, dia")
print("  - dia_da_semana_num (1=Domingo, 7=Sabado)")
print("  - dia_da_semana_nome (mapeamento em portugues)")
print("  - mes_nome (mapeamento em portugues)")
print("  - eh_fim_de_semana (Sim/Nao)")

print("\nSchema da dimensao:")
df_tempo.printSchema()

### Etapa 4: Gravacao da Tabela Delta

Persiste a dimensao tempo no formato Delta Lake no schema Gold com particionamento por ano e mes. O particionamento otimiza queries que filtram por periodos especificos, reduzindo a quantidade de dados lidos. A gravacao em modo overwrite garante que a dimensao seja recriada completamente a cada execucao, mantendo consistencia com os parametros de data_inicio e data_fim.

In [0]:
print("\n" + "="*80)
print("[ETAPA 3.3.4] Gravando gold.dm_tempo em Delta")
print("="*80)

ensure_database(gold_db_name)

target_table = f"{catalog_name}.{gold_db_name}.dm_tempo"
partition_cols = ["ano", "mes"]

print(f"\nTabela destino: {target_table}")
print(f"Particionamento: {', '.join(partition_cols)}")

total_a_gravar = df_tempo.count()
print(f"Registros a gravar: {total_a_gravar:,}")

print("\nGravando tabela Delta...")
write_delta_overwrite(df_tempo, target_table, partition_cols=partition_cols)

print("[ETAPA 33] Gravacao concluida")

### Etapa 5: Validacao da Dimensao Tempo

Valida a dimensao criada verificando contagem total de registros, datas minima e maxima, schema e tipos de dados. Exibe amostras de registros para verificacao visual da qualidade dos dados e dos mapeamentos de nomes em portugues. Calcula estatisticas de distribuicao por ano, trimestre e indicador de fim de semana para garantir que a dimensao foi criada corretamente e cobre todo o periodo especificado de forma uniforme.

In [0]:
print("\n" + "="*80)
print("[ETAPA 3.3.5] Validacao da dimensao tempo")
print("="*80)

df_validacao = spark.table(target_table)

count_validacao = df_validacao.count()
data_min = df_validacao.select(F.min("sk_tempo")).collect()[0][0]
data_max = df_validacao.select(F.max("sk_tempo")).collect()[0][0]

print(f"\nEstatisticas gerais:")
print(f"  Total de registros: {count_validacao:,}")
print(f"  Data minima: {data_min}")
print(f"  Data maxima: {data_max}")

if count_validacao != total_a_gravar:
    print(f"\n  ATENCAO: Divergencia de contagem (esperado: {total_a_gravar:,})")

print("\nSchema da tabela:")
df_validacao.printSchema()

print("\n[ETAPA 34] Amostra de registros (primeiros 15 dias):")
display(df_validacao.orderBy("sk_tempo").limit(15))

print("\n[ETAPA 34] Amostra de registros (ultimos 15 dias):")
display(df_validacao.orderBy(col("sk_tempo").desc()).limit(15))

print("\n[ETAPA 34] Distribuicao por ano:")
display(
    df_validacao.groupBy("ano")
    .agg(F.count("*").alias("total_dias"))
    .orderBy("ano")
)

print("\n[ETAPA 34] Distribuicao por trimestre (ano 2017):")
display(
    df_validacao.filter(col("ano") == 2017)
    .groupBy("trimestre")
    .agg(F.count("*").alias("total_dias"))
    .orderBy("trimestre")
)

print("\n[ETAPA 34] Distribuicao por dia da semana:")
display(
    df_validacao.groupBy("dia_da_semana_num", "dia_da_semana_nome", "eh_fim_de_semana")
    .agg(F.count("*").alias("total_dias"))
    .orderBy("dia_da_semana_num")
)

print("\n[ETAPA 34] Distribuicao por mes (ano 2017):")
display(
    df_validacao.filter(col("ano") == 2017)
    .groupBy("mes", "mes_nome")
    .agg(F.count("*").alias("total_dias"))
    .orderBy("mes")
)

### Etapa 6: Metricas Finais e Conclusao

Registra as metricas finais de criacao da dimensao tempo incluindo contagem total de registros, periodo coberto e status de conclusao. Esta etapa documenta o resultado da execucao e fornece confirmacao da criacao bem-sucedida da dimensao, que agora esta disponivel para uso em joins com tabelas fato para analises temporais de vendas.

In [0]:
metricas_dm_tempo = {
    "Tabela": target_table,
    "Total de registros": f"{count_validacao:,}",
    "Data inicio": str(data_min),
    "Data fim": str(data_max),
    "Particionamento": ", ".join(partition_cols),
    "Atributos": "ano, trimestre, mes, semana_do_ano, dia, dia_da_semana, mes_nome, eh_fim_de_semana",
    "Formato": "Delta Lake",
    "Status": "Criada com sucesso"
}

log_metricas("dm_tempo", metricas_dm_tempo)

print("[ETAPA 3.3.6] Dimensao de tempo gold.dm_tempo criada com sucesso")
print(f"[ETAPA 35] Registros: {count_validacao:,}, Inicio: {data_min}, Fim: {data_max}")

## Questao 3.2 - ft_vendas_geral (Tabela Fato Consolidada)

**Descricao:** Tabela fato unificada integrando todas as dimensoes e metricas de vendas, incluindo valores em BRL e USD, avaliacoes e metricas logisticas.

**Origem dos Dados:**
- silver.ft_pedidos (informacoes do pedido, status, datas)
- silver.ft_itens_pedidos (itens, valores produto e frete)
- silver.ft_consumidores (cliente)
- silver.dm_produtos (produto)
- silver.ft_avaliacoes_pedidos (avaliacoes agregadas por pedido)
- silver.ft_cotacao_dolar (conversao BRL para USD)

**Estrutura da Tabela:**
- id_pedido: Identificador do pedido (STRING)
- id_item: Identificador do item (STRING)
- fk_cliente: Chave estrangeira para dimensao cliente (STRING)
- fk_produto: Chave estrangeira para dimensao produto (STRING)
- fk_vendedor: Chave estrangeira para dimensao vendedor (STRING)
- fk_tempo: Data da compra para join com dm_tempo (DATE)
- status_pedido: Status do pedido (STRING)
- tempo_entrega_dias: Tempo real de entrega em dias (INT)
- entrega_no_prazo: Classificacao "Sim", "Nao" ou "Nao Entregue" (STRING)
- valor_produto_brl: Valor do produto em BRL (DECIMAL(12,2))
- valor_frete_brl: Valor do frete em BRL (DECIMAL(12,2))
- valor_total_item_brl: Soma produto + frete em BRL (DECIMAL(12,2))
- valor_produto_usd: Valor do produto em USD (DECIMAL(12,2))
- valor_frete_usd: Valor do frete em USD (DECIMAL(12,2))
- valor_total_item_usd: Soma produto + frete em USD (DECIMAL(12,2))
- cotacao_dolar: Cotacao do dolar utilizada na conversao (DECIMAL(8,4))
- avaliacao_pedido: Media das avaliacoes do pedido (DECIMAL(3,2))

**Regras de Negocio:**
- Conversao BRL para USD usa cotacao do dia do pedido
- Avaliacoes agregadas por pedido (media)
- Todos os valores financeiros arredondados com 2 casas decimais
- Integridade referencial garantida com todas as dimensoes

**Proposito:** Esta tabela fato e o nucleo do data warehouse Gold, consolidando todas as informacoes de vendas para analises multidimensionais. Permite analises por tempo, cliente, produto, vendedor, localidade, com metricas financeiras em multiplas moedas e indicadores de qualidade de servico.

### Etapa 1: Carregamento das Tabelas Silver

Carrega todas as tabelas Silver necessarias para construcao da tabela fato: ft_pedidos (dados do pedido), ft_itens_pedidos (itens e valores), ft_consumidores (clientes), dm_produtos (produtos), ft_avaliacoes_pedidos (avaliacoes) e ft_cotacao_dolar (taxas de cambio). Esta etapa estabelece a base de dados para a integracao multidimensional que comporao a fato consolidada de vendas.

In [0]:
print("="*80)
print("[ETAPA 3.4.1] Carregamento das tabelas Silver")
print("="*80)

df_pedidos_source = validar_tabela_silver("ft_pedidos")
df_itens_pedidos_source = validar_tabela_silver("ft_itens_pedidos")
df_consumidores_source = validar_tabela_silver("ft_consumidores")
df_produtos_source = validar_tabela_silver("ft_produtos")
df_avaliacoes_source = validar_tabela_silver("ft_avaliacoes_pedidos")
df_cotacao_source = validar_tabela_silver("dm_cotacao_dolar")

print("\n[ETAPA 01] Todas as tabelas Silver carregadas com sucesso")

### Etapa 2: Agregacao de Avaliacoes por Pedido

Calcula a media das avaliacoes por pedido a partir da tabela ft_avaliacoes_pedidos. Como um pedido pode ter multiplas avaliacoes ao longo do tempo, esta agregacao consolida a nota media final de cada pedido. A metrica de avaliacao e essencial para analises de satisfacao do cliente e qualidade do servico, permitindo correlacoes com outras dimensoes como vendedor, produto e tempo de entrega.

In [0]:
print("\n" + "="*80)
print("[ETAPA 3.4.2] Agregacao de avaliacoes por pedido")
print("="*80)

df_avaliacoes_agg = df_avaliacoes_source.groupBy("id_pedido").agg(
    spark_round(avg("avaliacao"), 2).alias("avaliacao_pedido")
)

total_pedidos_avaliados = df_avaliacoes_agg.count()
print(f"\nPedidos com avaliacao: {total_pedidos_avaliados:,}")

print("\n[ETAPA 02] Agregacao concluida")

### Etapa 3: Preparacao da Base de Itens de Pedidos

Prepara o dataframe principal a partir de ft_itens_pedidos, que contem a granularidade item-a-item necessaria para a tabela fato. Seleciona e tipifica as colunas essenciais incluindo identificadores (pedido, item, produto, vendedor) e valores financeiros (produto e frete em BRL). Esta base sera enriquecida com informacoes de outras tabelas atraves de joins sucessivos, mantendo a granularidade de item como nivel atomico da fato.

In [0]:
print("\n" + "="*80)
print("[ETAPA 3.4.3] Preparacao da base de itens")
print("="*80)

df_base = df_itens_pedidos_source.select(
    col("id_pedido").cast("string").alias("id_pedido"),
    col("id_item").cast("string").alias("id_item"),
    col("id_produto").cast("string").alias("fk_produto"),
    col("id_vendedor").cast("string").alias("fk_vendedor"),
    spark_round(col("preco_brl"), 2).alias("valor_produto_brl"),
    spark_round(col("preco_frete"), 2).alias("valor_frete_brl")
)

total_itens = df_base.count()
print(f"\nTotal de itens processados: {total_itens:,}")

print("\n[ETAPA 03] Base preparada")

### Etapa 4: Enriquecimento com Dados de Pedidos

Executa join left entre a base de itens e ft_pedidos para adicionar informacoes do pedido: cliente (fk_cliente), data da compra (fk_tempo), status, metricas de entrega (tempo_entrega_dias, entrega_no_prazo) e diferenca de entrega. O join left garante que todos os itens sejam mantidos mesmo se houver inconsistencias nos dados de pedido. A data de compra e essencial para posterior join com cotacao do dolar e dimensao tempo.

In [0]:
print("\n" + "="*80)
print("[ETAPA 3.4.4] Enriquecimento com dados de pedidos")
print("="*80)

df_pedidos_prep = df_pedidos_source.select(
    col("id_pedido").cast("string").alias("id_pedido"),
    col("id_consumidor").cast("string").alias("fk_cliente"),
    to_date(col("pedido_compra_timestamp")).alias("fk_tempo"),
    col("status").alias("status_pedido"),
    col("diferenca_entrega_dias").cast("int").alias("tempo_entrega_dias"),
    col("entrega_no_prazo")
)

df_enriched = df_base.join(
    df_pedidos_prep,
    on="id_pedido",
    how="left"
)

total_enriched = df_enriched.count()
print(f"\nRegistros apos join com pedidos: {total_enriched:,}")

print("\n[ETAPA 04] Enriquecimento concluido")

### Etapa 5: Adicao de Avaliacoes

Executa join left entre o dataframe enriquecido e as avaliacoes agregadas por pedido. O join left garante que itens de pedidos sem avaliacao sejam mantidos com valor nulo. A metrica de avaliacao permite analises de correlacao entre satisfacao do cliente e outras dimensoes como vendedor, produto, tempo de entrega e localidade, sendo fundamental para estrategias de melhoria de qualidade.

In [0]:
print("\n" + "="*80)
print("[ETAPA 3.4.5] Adicao de avaliacoes")
print("="*80)

df_with_avaliacoes = df_enriched.join(
    df_avaliacoes_agg,
    on="id_pedido",
    how="left"
)

total_with_avaliacoes = df_with_avaliacoes.count()
itens_com_avaliacao = df_with_avaliacoes.filter(col("avaliacao_pedido").isNotNull()).count()

print(f"\nRegistros totais: {total_with_avaliacoes:,}")
print(f"Itens com avaliacao: {itens_com_avaliacao:,}")
print(f"Itens sem avaliacao: {total_with_avaliacoes - itens_com_avaliacao:,}")

print("\n[ETAPA 05] Avaliacoes adicionadas")

### Etapa 6: Conversao de Valores para USD

Executa join left com ft_cotacao_dolar usando fk_tempo (data da compra) para obter a cotacao do dolar do dia. Calcula valores em USD dividindo valores BRL pela cotacao e arredondando para 2 casas decimais. Calcula tambem o valor total do item (produto + frete) em ambas as moedas. A conversao para USD permite analises comparativas internacionais e protege contra distorcoes inflacionarias, sendo essencial para analises de tendencias de longo prazo.

In [0]:
print("\n" + "="*80)
print("[ETAPA 3.4.6] Conversao de valores para USD")
print("="*80)

df_cotacao_prep = df_cotacao_source.select(
    to_date(col("data")).alias("data_cotacao"),
    col("cotacao_dolar").cast(DecimalType(8,4)).alias("cotacao_dolar")
)

df_with_cotacao = df_with_avaliacoes.join(
    df_cotacao_prep,
    df_with_avaliacoes.fk_tempo == df_cotacao_prep.data_cotacao,
    how="left"
).drop("data_cotacao")

df_with_usd = df_with_cotacao.withColumn(
    "valor_total_item_brl",
    spark_round(col("valor_produto_brl") + col("valor_frete_brl"), 2)
).withColumn(
    "valor_produto_usd",
    spark_round(col("valor_produto_brl") / col("cotacao_dolar"), 2)
).withColumn(
    "valor_frete_usd",
    spark_round(col("valor_frete_brl") / col("cotacao_dolar"), 2)
).withColumn(
    "valor_total_item_usd",
    spark_round((col("valor_produto_brl") + col("valor_frete_brl")) / col("cotacao_dolar"), 2)
)

total_with_usd = df_with_usd.count()
itens_com_cotacao = df_with_usd.filter(col("cotacao_dolar").isNotNull()).count()

print(f"\nRegistros totais: {total_with_usd:,}")
print(f"Itens com cotacao: {itens_com_cotacao:,}")
print(f"Itens sem cotacao: {total_with_usd - itens_com_cotacao:,}")

print("\n[ETAPA 06] Conversao concluida")

### Etapa 7: Selecao Final de Colunas

Seleciona e organiza as colunas finais na ordem especificada para a tabela fato, garantindo tipagem correta e nomenclatura padronizada. Esta etapa finaliza a estrutura da tabela fato com todas as chaves estrangeiras (fk_cliente, fk_produto, fk_vendedor, fk_tempo), metricas de negocio (valores financeiros em BRL e USD, tempo de entrega) e indicadores de qualidade (avaliacao, entrega no prazo). A tabela resultante esta pronta para validacoes de integridade antes da persistencia.

In [0]:
print("\n" + "="*80)
print("[ETAPA 3.4.7] Selecao final de colunas")
print("="*80)

df_fato = df_with_usd.withColumn("ano", year(col("fk_tempo"))).select(
    col("id_pedido"),
    col("id_item"),
    col("fk_cliente"),
    col("fk_produto"),
    col("fk_vendedor"),
    col("fk_tempo"),
    col("status_pedido"),
    col("tempo_entrega_dias"),
    col("entrega_no_prazo"),
    col("valor_produto_brl"),
    col("valor_frete_brl"),
    col("valor_total_item_brl"),
    col("valor_produto_usd"),
    col("valor_frete_usd"),
    col("valor_total_item_usd"),
    col("cotacao_dolar"),
    col("avaliacao_pedido"),
    col("ano")
)

total_fato = df_fato.count()
print(f"\nRegistros na tabela fato: {total_fato:,}")

print("\n[ETAPA 07] Estrutura final criada")

### Etapa 8: Validacoes de Integridade

Executa validacoes criticas de qualidade antes da persistencia: verifica chaves nulas (id_pedido, id_item, fk_cliente, fk_produto, fk_vendedor, fk_tempo), detecta duplicatas na chave composta (id_pedido, id_item) e valida valores financeiros anomalos. Estas validacoes garantem a integridade referencial da tabela fato e identificam problemas de qualidade de dados que podem comprometer analises downstream. Falhas nessas validacoes indicam necessidade de tratamento na camada Silver.

In [0]:
print("\n" + "="*80)
print("[ETAPA 3.4.8] Validacoes de integridade")
print("="*80)

print("\nValidacao 1: Chaves nulas")
null_id_pedido = df_fato.filter(col("id_pedido").isNull()).count()
null_id_item = df_fato.filter(col("id_item").isNull()).count()
null_fk_cliente = df_fato.filter(col("fk_cliente").isNull()).count()
null_fk_produto = df_fato.filter(col("fk_produto").isNull()).count()
null_fk_vendedor = df_fato.filter(col("fk_vendedor").isNull()).count()
null_fk_tempo = df_fato.filter(col("fk_tempo").isNull()).count()

print(f"  id_pedido nulo: {null_id_pedido}")
print(f"  id_item nulo: {null_id_item}")
print(f"  fk_cliente nulo: {null_fk_cliente}")
print(f"  fk_produto nulo: {null_fk_produto}")
print(f"  fk_vendedor nulo: {null_fk_vendedor}")
print(f"  fk_tempo nulo: {null_fk_tempo}")

if any([null_id_pedido, null_id_item, null_fk_cliente, null_fk_produto, null_fk_vendedor, null_fk_tempo]):
    print("  ATENCAO: Chaves nulas detectadas")
else:
    print("  OK: Nenhuma chave nula")

print("\nValidacao 2: Duplicatas na chave composta")
total_registros = df_fato.count()
registros_distintos = df_fato.select("id_pedido", "id_item").distinct().count()
duplicatas = total_registros - registros_distintos

print(f"  Total de registros: {total_registros:,}")
print(f"  Registros distintos: {registros_distintos:,}")
print(f"  Duplicatas: {duplicatas}")

if duplicatas > 0:
    print("  ATENCAO: Duplicatas detectadas na chave (id_pedido, id_item)")
else:
    print("  OK: Nenhuma duplicata")

print("\nValidacao 3: Valores financeiros")
valores_negativos = df_fato.filter(
    (col("valor_produto_brl") < 0) | 
    (col("valor_frete_brl") < 0) |
    (col("valor_total_item_brl") < 0)
).count()

valores_nulos = df_fato.filter(
    col("valor_produto_brl").isNull() |
    col("valor_frete_brl").isNull() |
    col("valor_total_item_brl").isNull()
).count()

print(f"  Valores negativos: {valores_negativos}")
print(f"  Valores nulos: {valores_nulos}")

if valores_negativos > 0 or valores_nulos > 0:
    print("  ATENCAO: Valores anomalos detectados")
else:
    print("  OK: Todos os valores financeiros validos")

print("\n[ETAPA 08] Validacoes concluidas")

### Etapa 9: Gravacao da Tabela Fato Gold

Persiste a tabela fato ft_vendas_geral no formato Delta Lake no schema Gold. A tabela nao utiliza particionamento fisico pois a granularidade item-a-item e as multiplas dimensoes de analise tornariam o particionamento ineficiente. O Delta Lake fornece otimizacoes automaticas (Z-ordering, data skipping) que sao mais adequadas para este caso de uso. A gravacao em modo overwrite garante substituicao completa dos dados a cada execucao.

In [0]:
print("\n" + "="*80)
print("[ETAPA 3.4.9] Gravacao da tabela fato Gold")
print("="*80)

ensure_database(gold_db_name)

target_table = f"{catalog_name}.{gold_db_name}.ft_vendas_geral"

print(f"\nTabela destino: {target_table}")

total_a_gravar = df_fato.count()
print(f"Registros a gravar: {total_a_gravar:,}")

print("\nGravando tabela Delta...")
write_delta_overwrite(df_fato, target_table, partition_cols=["ano"])

print("\n[ETAPA 09] Gravacao concluida")

### Etapa 10: Validacao Pos-Gravacao e Visualizacao

Valida a tabela gravada verificando contagem de registros, schema, tipos de dados e distribuicoes. Exibe amostras dos dados para verificacao visual da qualidade e consistencia das transformacoes. Calcula estatisticas agregadas por dimensoes chave (status, entrega no prazo, periodo) para confirmar a integridade dos dados e fornecer insights preliminares. Esta etapa final confirma que a tabela fato esta pronta para uso em analises e dashboards.

In [0]:
print("\n" + "="*80)
print("[ETAPA 3.4.10] Validacao pos-gravacao")
print("="*80)

df_validacao = spark.table(target_table)
count_validacao = df_validacao.count()

print(f"\nRegistros gravados: {count_validacao:,}")

if count_validacao != total_a_gravar:
    print(f"ATENCAO: Divergencia de contagem (esperado: {total_a_gravar:,})")
else:
    print("OK: Contagem validada")

print("\nSchema da tabela:")
df_validacao.printSchema()

print("\n[ETAPA 10] Amostra dos dados (20 registros):")
display(df_validacao.limit(20))

print("\n[ETAPA 10] Estatisticas por status de pedido:")
display(
    df_validacao.groupBy("status_pedido")
    .agg(
        F.count("*").alias("total_itens"),
        spark_round(F.sum("valor_total_item_brl"), 2).alias("valor_total_brl"),
        spark_round(F.sum("valor_total_item_usd"), 2).alias("valor_total_usd"),
        spark_round(F.avg("avaliacao_pedido"), 2).alias("avaliacao_media")
    )
    .orderBy(col("total_itens").desc())
)

print("\n[ETAPA 10] Estatisticas por entrega no prazo:")
display(
    df_validacao.groupBy("entrega_no_prazo")
    .agg(
        F.count("*").alias("total_itens"),
        spark_round(F.avg("tempo_entrega_dias"), 2).alias("tempo_medio_entrega"),
        spark_round(F.avg("avaliacao_pedido"), 2).alias("avaliacao_media")
    )
    .orderBy("entrega_no_prazo")
)

print("\n[ETAPA 10] Estatisticas por ano:")
display(
    df_validacao.withColumn("ano", year(col("fk_tempo")))
    .groupBy("ano")
    .agg(
        F.count("*").alias("total_itens"),
        spark_round(F.sum("valor_total_item_brl"), 2).alias("valor_total_brl"),
        spark_round(F.sum("valor_total_item_usd"), 2).alias("valor_total_usd")
    )
    .orderBy("ano")
)

print("\n[ETAPA 10] Cobertura de dados:")
total_registros = df_validacao.count()
com_avaliacao = df_validacao.filter(col("avaliacao_pedido").isNotNull()).count()
com_cotacao = df_validacao.filter(col("cotacao_dolar").isNotNull()).count()

print(f"  Total de itens: {total_registros:,}")
print(f"  Com avaliacao: {com_avaliacao:,} ({100*com_avaliacao/total_registros:.1f}%)")
print(f"  Com cotacao USD: {com_cotacao:,} ({100*com_cotacao/total_registros:.1f}%)")

metricas_fato = {
    "Tabela": target_table,
    "Total de itens": f"{count_validacao:,}",
    "Origem": "ft_pedidos, ft_itens_pedidos, ft_consumidores, dm_produtos, ft_avaliacoes_pedidos, ft_cotacao_dolar",
    "Chaves estrangeiras": "fk_cliente, fk_produto, fk_vendedor, fk_tempo",
    "Metricas": "valores BRL/USD, tempo entrega, avaliacao",
    "Formato": "Delta Lake",
    "Status": "Criada com sucesso"
}

log_metricas("ft_vendas_geral", metricas_fato)

print("\n[ETAPA 10] Tabela fato gold.ft_vendas_geral criada e validada com sucesso")

## Questao 3.3 - view_vendas_por_periodo

**Descricao:** View analitica consolidando vendas por diferentes granularidades temporais (ano, trimestre, mes, dia da semana) para analises de sazonalidade e tendencias.

**Origem dos Dados:**
- gold.ft_vendas_geral (metricas de vendas)
- gold.dm_tempo (dimensao tempo para consistencia de nomes)

**Estrutura da View:**
- ano: Ano da venda (INT)
- trimestre: Trimestre do ano, valores 1 a 4 (INT)
- mes: Mes do ano, valores 1 a 12 (INT)
- mes_nome: Nome do mes em portugues (STRING)
- dia: Dia do mes (INT)
- dia_da_semana_num: Dia da semana numerico, 1=Domingo, 7=Sabado (INT)
- total_pedidos: Quantidade de pedidos distintos (BIGINT)
- total_itens: Quantidade total de itens vendidos (BIGINT)
- receita_total_brl: Receita total em BRL (DECIMAL(12,2))
- receita_total_usd: Receita total em USD (DECIMAL(12,2))
- ticket_medio_brl: Ticket medio por item em BRL (DECIMAL(12,2))
- avaliacao_media: Media das avaliacoes de pedidos entregues/enviados (DECIMAL(3,2))

**Regras de Negocio:**
- ticket_medio_brl = receita_total_brl / total_itens
- avaliacao_media considera apenas pedidos com status 'entregue' ou 'enviado'
- Join com dm_tempo garante consistencia de nomes e numeracao de dias
- Todos os valores arredondados com 2 casas decimais

**Proposito:** Esta view permite analises temporais multidimensionais de vendas, identificando padroes de sazonalidade por diferentes granularidades (anual, trimestral, mensal, semanal). Essencial para previsao de demanda, planejamento de estoque e estrategias comerciais baseadas em ciclos temporais.

### Etapa 1: Carregamento das Tabelas Gold

Carrega as tabelas gold.ft_vendas_geral (fonte das metricas de vendas) e gold.dm_tempo (dimensao tempo para garantir consistencia de nomes de meses e dias da semana). O join entre essas tabelas assegura que os atributos temporais estejam normalizados conforme a dimensao tempo, mantendo coerencia entre diferentes analises temporais no data warehouse.

In [0]:
print("="*80)
print("[ETAPA 3.5.1] Lendo tabelas Gold")
print("="*80)

df_vendas = spark.table(f"{catalog_name}.{gold_db_name}.ft_vendas_geral")
df_tempo = spark.table(f"{catalog_name}.{gold_db_name}.dm_tempo")

total_itens_vendas = df_vendas.count()
total_datas_tempo = df_tempo.count()

print(f"\nft_vendas_geral: {total_itens_vendas:,} registros")
print(f"dm_tempo: {total_datas_tempo:,} registros")

print("\n[ETAPA 3.5.1] Tabelas carregadas")

### Etapa 2: Filtragem e Preparacao dos Dados

Filtra registros da tabela fato removendo linhas com chaves nulas (fk_tempo, id_pedido, id_item) que comprometem a integridade da analise temporal. Executa join left com dm_tempo usando fk_tempo = sk_tempo para enriquecer os dados com atributos temporais normalizados (mes_nome, dia_da_semana_num). Esta preparacao garante qualidade e consistencia dos dados antes da agregacao.

In [0]:
print("\n" + "="*80)
print("[ETAPA 3.5.2] Filtragem e join com dimensao tempo")
print("="*80)

df_vendas_filtrado = df_vendas.filter(
    col("fk_tempo").isNotNull() &
    col("id_pedido").isNotNull() &
    col("id_item").isNotNull()
)

total_antes = df_vendas.count()
total_depois = df_vendas_filtrado.count()
registros_removidos = total_antes - total_depois

print(f"\nRegistros antes do filtro: {total_antes:,}")
print(f"Registros apos filtro: {total_depois:,}")
print(f"Registros removidos: {registros_removidos:,}")

df_tempo_prep = df_tempo.select(
    col("sk_tempo"),
    col("mes_nome"),
    col("dia_da_semana_num")
)

df_vendas_enriched = df_vendas_filtrado.join(
    df_tempo_prep,
    df_vendas_filtrado.fk_tempo == df_tempo_prep.sk_tempo,
    how="left"
).drop("sk_tempo")

total_enriched = df_vendas_enriched.count()
print(f"\nRegistros apos join com dm_tempo: {total_enriched:,}")

print("\n[ETAPA 3.5.2] Dados preparados")

### Etapa 3: Agregacao por Dimensoes Temporais

Agrega os dados por ano, trimestre, mes, dia e dia da semana, calculando as metricas essenciais: total de pedidos distintos, total de itens, receitas em BRL e USD, ticket medio por item e avaliacao media. A avaliacao media e filtrada para considerar apenas pedidos com status 'entregue' ou 'enviado', refletindo a satisfacao real do cliente com pedidos completados. Todas as metricas financeiras sao arredondadas para 2 casas decimais.

In [0]:
print("\n" + "="*80)
print("[ETAPA 3.5.3] Agregando por ano, trimestre, mes, dia da semana")
print("="*80)

df_agregado = df_vendas_enriched.groupBy(
    year(col("fk_tempo")).alias("ano"),
    quarter(col("fk_tempo")).alias("trimestre"),
    month(col("fk_tempo")).alias("mes"),
    col("mes_nome"),
    dayofmonth(col("fk_tempo")).alias("dia"),
    col("dia_da_semana_num")
).agg(
    F.countDistinct("id_pedido").alias("total_pedidos"),
    F.count("id_item").alias("total_itens"),
    spark_round(F.sum("valor_total_item_brl"), 2).alias("receita_total_brl"),
    spark_round(F.sum("valor_total_item_usd"), 2).alias("receita_total_usd"),
    spark_round(
        F.avg(
            when(
                (col("status_pedido") == "entregue") | (col("status_pedido") == "enviado"),
                col("avaliacao_pedido")
            )
        ), 2
    ).alias("avaliacao_media")
)

df_view = df_agregado.withColumn(
    "ticket_medio_brl",
    spark_round(
        when(col("total_itens") > 0, col("receita_total_brl") / col("total_itens"))
        .otherwise(lit(0)), 
        2
    )
)

total_grupos = df_view.count()
print(f"\nGrupos temporais criados: {total_grupos:,}")

print("\n[ETAPA 3.5.3] Agregacao concluida")

### Etapa 4: Criacao da View Gold

Persiste o dataframe agregado como view gold.view_vendas_por_periodo no formato Delta. A view consolida vendas por multiplas granularidades temporais (ano, trimestre, mes, dia, dia da semana) com metricas essenciais de receita, volume e satisfacao. Esta estrutura permite analises rapidas de sazonalidade, tendencias e padroes de vendas sem necessidade de reprocessamento da tabela fato completa.

In [0]:
print("\n" + "="*80)
print("[ETAPA 3.5.4] Criando view gold.view_vendas_por_periodo")
print("="*80)

view_name = f"{catalog_name}.{gold_db_name}.view_vendas_por_periodo"

print(f"\nView: {view_name}")

df_view.write.format("delta").mode("overwrite").saveAsTable(view_name)

print("\n[ETAPA 3.5.4] View criada com sucesso")

### Etapa 5: Validacao da View

Valida a view criada verificando contagem total de registros, intervalo de anos cobertos (minimo e maximo), schema e tipos de dados. Exibe amostras dos dados para inspecao visual da qualidade das agregacoes. Valida a consistencia das metricas calculadas verificando que ticket_medio_brl = receita_total_brl / total_itens em amostras. Calcula estatisticas agregadas por ano e dia da semana para confirmar a distribuicao temporal dos dados.

In [0]:
print("\n" + "="*80)
print("[ETAPA 3.5.5] Validacao da view")
print("="*80)

df_view_read = spark.table(view_name)

count_view = df_view_read.count()
ano_min = df_view_read.select(F.min("ano")).collect()[0][0]
ano_max = df_view_read.select(F.max("ano")).collect()[0][0]

print(f"\nTotal de registros na view: {count_view:,}")
print(f"Periodo coberto: {ano_min} a {ano_max}")

print("\nSchema da view:")
df_view_read.printSchema()

print("\n[ETAPA 45] Amostra dos dados (20 registros ordenados por ano/mes/dia):")
display(
    df_view_read.orderBy("ano", "mes", "dia")
    .limit(20)
)

print("\n[ETAPA 45] Validacao do ticket medio (amostra):")
display(
    df_view_read.withColumn(
        "ticket_calculado",
        spark_round(col("receita_total_brl") / col("total_itens"), 2)
    ).withColumn(
        "diferenca",
        spark_round(col("ticket_medio_brl") - col("ticket_calculado"), 2)
    ).select(
        "ano", "mes", "dia",
        "receita_total_brl", "total_itens",
        "ticket_medio_brl", "ticket_calculado", "diferenca"
    ).orderBy("ano", "mes", "dia")
    .limit(10)
)

print("\n[ETAPA 45] Agregacao por ano:")
display(
    df_view_read.groupBy("ano")
    .agg(
        F.sum("total_pedidos").alias("pedidos_ano"),
        F.sum("total_itens").alias("itens_ano"),
        spark_round(F.sum("receita_total_brl"), 2).alias("receita_ano_brl"),
        spark_round(F.sum("receita_total_usd"), 2).alias("receita_ano_usd")
    )
    .orderBy("ano")
)

print("\n[ETAPA 45] Agregacao por dia da semana:")
display(
    df_view_read.groupBy("dia_da_semana_num")
    .agg(
        F.sum("total_pedidos").alias("pedidos_dia_semana"),
        F.sum("total_itens").alias("itens_dia_semana"),
        spark_round(F.sum("receita_total_brl"), 2).alias("receita_dia_semana_brl"),
        spark_round(F.avg("avaliacao_media"), 2).alias("avaliacao_media_geral")
    )
    .orderBy("dia_da_semana_num")
)

print("\n[ETAPA 45] Agregacao por mes (ano 2017):")
display(
    df_view_read.filter(col("ano") == 2017)
    .groupBy("mes", "mes_nome")
    .agg(
        F.sum("total_pedidos").alias("pedidos_mes"),
        F.sum("total_itens").alias("itens_mes"),
        spark_round(F.sum("receita_total_brl"), 2).alias("receita_mes_brl")
    )
    .orderBy("mes")
)

metricas_view = {
    "View": view_name,
    "Total de registros": f"{count_view:,}",
    "Periodo": f"{ano_min} a {ano_max}",
    "Origem": "ft_vendas_geral, dm_tempo",
    "Granularidades": "ano, trimestre, mes, dia, dia_da_semana",
    "Metricas": "pedidos, itens, receitas BRL/USD, ticket medio, avaliacao",
    "Status": "Criada e validada com sucesso"
}

log_metricas("view_vendas_por_periodo", metricas_view)

print("\n[ETAPA 45] View gold.view_vendas_por_periodo validada com sucesso")

### Questao 3.3.1 - Consultas Analiticas

**Objetivo:** Executar consultas analiticas solicitadas pela equipe Comercial para avaliar padroes de vendas e sazonalidade usando exclusivamente a view gold.view_vendas_por_periodo.

**Consultas:**
1. Dia da semana com maior receita total em BRL
2. Mes com maior ticket medio no ultimo ano disponivel

**Proposito:** Estas consultas demonstram o uso da view agregada para responder rapidamente perguntas de negocio sobre padroes temporais de vendas, sem necessidade de processar a tabela fato completa. Identificam oportunidades de otimizacao de operacoes e estrategias comerciais baseadas em sazonalidade.

#### Consulta 1: Dia da Semana com Maior Receita

Identifica qual dia da semana gera maior receita total em BRL ao longo de todo o periodo analisado. Agrega a receita por dia da semana somando todas as ocorrencias daquele dia, independente do ano, mes ou data especifica. Esta analise revela padroes semanais de vendas que podem direcionar estrategias de marketing, promocoes e gestao de estoque conforme a demanda esperada em cada dia da semana.

In [0]:
print("="*80)
print("[ETAPA 3.5.1.1] Calculando receita por dia da semana")
print("="*80)

df_view_vendas = spark.table(f"{catalog_name}.{gold_db_name}.view_vendas_por_periodo")

df_tempo_nomes = spark.table(f"{catalog_name}.{gold_db_name}.dm_tempo").select(
    "dia_da_semana_num",
    "dia_da_semana_nome"
).distinct()

df_receita_dia_semana = df_view_vendas.groupBy(
    "dia_da_semana_num"
).agg(
    spark_round(F.sum("receita_total_brl"), 2).alias("receita_total_brl")
).join(
    df_tempo_nomes,
    on="dia_da_semana_num",
    how="left"
).select(
    "dia_da_semana_num",
    "dia_da_semana_nome",
    "receita_total_brl"
).orderBy(col("receita_total_brl").desc())

print("\n[ETAPA 51] Receita por dia da semana (ordenado por maior receita):")
display(df_receita_dia_semana)

dia_maior_receita = df_receita_dia_semana.first()
print(f"\n[ETAPA 51] Dia com maior receita: {dia_maior_receita['dia_da_semana_nome']} (#{dia_maior_receita['dia_da_semana_num']}) - R$ {dia_maior_receita['receita_total_brl']:,.2f}")

#### Consulta 2: Mes com Maior Ticket Medio no Ultimo Ano

Identifica o mes com maior ticket medio de vendas no ultimo ano disponivel nos dados. Primeiro determina automaticamente qual e o ultimo ano com vendas registradas, depois agrega os dados por mes dentro daquele ano especifico. O ticket medio revela o valor medio gasto por item, sendo um indicador importante da qualidade das vendas e efetividade de estrategias de upselling e cross-selling.

In [0]:
print("\n" + "="*80)
print("[ETAPA 3.5.1.2] Buscando ultimo ano disponivel")
print("="*80)

ultimo_ano = df_view_vendas.select(F.max("ano")).collect()[0][0]

print(f"\nUltimo ano com dados: {ultimo_ano}")

print("\n[ETAPA 52] Ultimo ano identificado")

#### Etapa 2: Agregacao por Mes no Ultimo Ano

Filtra os dados para o ultimo ano identificado e agrega por mes, calculando o ticket medio total de cada mes. Como a view ja contem o ticket_medio_brl pre-calculado por dia, precisamos recalcular a media ponderada considerando o volume de itens de cada dia dentro do mes. Ordena os resultados para identificar qual mes apresentou o maior valor medio por item vendido.

In [0]:
print("\n" + "="*80)
print("[ETAPA 3.5.1.3] Calculando ticket medio por mes no ultimo ano")
print("="*80)

df_ultimo_ano = df_view_vendas.filter(col("ano") == ultimo_ano)

df_ticket_mes = df_ultimo_ano.groupBy(
    "mes",
    "mes_nome"
).agg(
    F.sum("receita_total_brl").alias("receita_mes"),
    F.sum("total_itens").alias("itens_mes")
).withColumn(
    "ticket_medio_brl",
    spark_round(
        when(col("itens_mes") > 0, col("receita_mes") / col("itens_mes"))
        .otherwise(lit(0)),
        2
    )
).select(
    "mes",
    "mes_nome",
    "ticket_medio_brl"
).orderBy(col("ticket_medio_brl").desc())

print(f"\n[ETAPA 53] Ticket medio por mes no ano {ultimo_ano} (ordenado por maior ticket):")
display(df_ticket_mes)

mes_maior_ticket = df_ticket_mes.first()
print(f"\n[ETAPA 53] Mes com maior ticket medio: {mes_maior_ticket['mes_nome']} (#{mes_maior_ticket['mes']}) - R$ {mes_maior_ticket['ticket_medio_brl']:,.2f}")

## Questao 3.4 - view_top_produto

**Descricao:** View analitica consolidando metricas de desempenho por produto e categoria para analises de gestao de produtos e estrategias comerciais.

**Origem dos Dados:**
- gold.ft_vendas_geral (vendas, receitas, avaliacoes)
- silver.ft_produtos (informacoes de catalogo: categoria, peso)

**Estrutura da View:**
- id_produto: Identificador do produto (STRING)
- categoria_produto: Categoria do produto (STRING)
- quantidade_vendida: Total de unidades vendidas (BIGINT)
- total_pedidos: Quantidade de pedidos distintos contendo o produto (BIGINT)
- receita_brl: Receita total gerada em BRL (DECIMAL(12,2))
- receita_usd: Receita total gerada em USD (DECIMAL(12,2))
- preco_medio_brl: Preco medio unitario em BRL (DECIMAL(12,2))
- avaliacao_media: Media das avaliacoes de pedidos contendo o produto (DECIMAL(3,2))
- peso_medio_gramas: Peso medio do produto em gramas (DECIMAL(8,2))

**Regras de Negocio:**
- quantidade_vendida = count(id_item) por produto
- total_pedidos = countDistinct(id_pedido) por produto
- receita_brl = sum(valor_produto_brl)
- receita_usd = sum(valor_produto_usd)
- preco_medio_brl = receita_brl / quantidade_vendida
- avaliacao_media = avg(avaliacao_pedido) com filtro de pedidos validos
- peso_medio_gramas obtido do catalogo ft_produtos

**Proposito:** Esta view permite a equipe de Gestao de Produtos identificar produtos e categorias de maior performance, avaliar estrategias de precificacao, detectar produtos com baixa avaliacao que requerem atencao e otimizar mix de produtos baseado em receita e demanda. Essencial para decisoes de descontinuacao, investimento e promocoes direcionadas.

### Etapa 1: Carregamento das Tabelas

Carrega a tabela fato gold.ft_vendas_geral (fonte das metricas de vendas) e a tabela silver.ft_produtos (catalogo de produtos com informacoes de categoria e peso). O join entre essas tabelas permite enriquecer as metricas de vendas com atributos descritivos dos produtos essenciais para analises de gestao de catalogo.

In [0]:
print("="*80)
print("[ETAPA 3.6.1] Carregamento das tabelas")
print("="*80)

df_vendas_geral = spark.table(f"{catalog_name}.{gold_db_name}.ft_vendas_geral")
df_produtos_cat = validar_tabela_silver("ft_produtos")

total_itens_vendas = df_vendas_geral.count()
total_produtos_catalogo = df_produtos_cat.count()

print(f"\nft_vendas_geral: {total_itens_vendas:,} registros")
print(f"ft_produtos: {total_produtos_catalogo:,} registros")

print("\n[ETAPA 3.6.1] Tabelas carregadas")

### Etapa 2: Validacao de Integridade

Valida a integridade dos dados verificando produtos nulos na tabela fato e produtos orfaos (presentes nas vendas mas ausentes no catalogo). Produtos orfaos indicam problemas de sincronizacao entre sistemas ou exclusoes incorretas do catalogo. Esta validacao identifica inconsistencias que podem comprometer analises de categoria e atributos de produtos.

In [0]:
print("[ETAPA 62] Validacao de integridade dos dados")

produtos_nulos_vendas = df_vendas_geral.filter(col("fk_produto").isNull()).count()
print(f"\nProdutos nulos em ft_vendas_geral: {produtos_nulos_vendas:,}")

produtos_vendas_unicos = df_vendas_geral.select("fk_produto").distinct()
total_produtos_vendidos = produtos_vendas_unicos.count()
print(f"Total de produtos unicos vendidos: {total_produtos_vendidos:,}")

produtos_catalogo_ids = df_produtos_cat.select(col("id_produto").alias("fk_produto"))
produtos_orfaos = produtos_vendas_unicos.join(produtos_catalogo_ids, "fk_produto", "left_anti")
total_produtos_orfaos = produtos_orfaos.count()

if total_produtos_orfaos > 0:
    print(f"\nALERTA: {total_produtos_orfaos:,} produtos vendidos ausentes no catalogo")
else:
    print(f"\nOK: Todos os produtos vendidos existem no catalogo")

print("\n[ETAPA 62] Validacao concluida")

### Etapa 3: Agregacao por Produto

Agrega as vendas por produto calculando as metricas fundamentais: quantidade total vendida (count de itens), numero de pedidos distintos (countDistinct de id_pedido), receita total em BRL e USD (sum dos valores), e avaliacao media (avg das notas). Estas metricas formam a base para identificar produtos top performance e avaliar estrategias de pricing.

In [0]:
print("[ETAPA 3.6.3] Agregacao das metricas por produto")

df_produto_agg = df_vendas_geral.groupBy("fk_produto").agg(
    F.count("id_item").alias("quantidade_vendida"),
    F.countDistinct("id_pedido").alias("total_pedidos"),
    spark_round(F.sum("valor_produto_brl"), 2).alias("receita_brl"),
    spark_round(F.sum("valor_produto_usd"), 2).alias("receita_usd"),
    spark_round(F.avg(when(col("avaliacao_pedido").isNotNull(), col("avaliacao_pedido"))), 2).alias("avaliacao_media")
)

total_produtos_agg = df_produto_agg.count()
print(f"\nTotal de produtos agregados: {total_produtos_agg:,}")

print("\nAmostra dos top 3 produtos por receita BRL:")
df_sample_top3 = df_produto_agg.orderBy(col("receita_brl").desc()).limit(3)
display(df_sample_top3)

print("\n[ETAPA 63] Agregacao concluida")

### Etapa 4: Enriquecimento com Catalogo de Produtos

Realiza join entre as metricas agregadas e o catalogo de produtos para adicionar os atributos descritivos: categoria do produto (categoria_produto_ingles traduzida) e peso medio em gramas (peso_produto_gramas). O join left preserva todos os produtos vendidos mesmo que ausentes no catalogo (produtos orfaos identificados na Etapa 2).

In [0]:
print("[ETAPA 3.6.4] Enriquecimento com atributos do catalogo")

df_produtos_prep = df_produtos_cat.select(
    col("id_produto"),
    col("categoria_produto"),
    col("peso_produto_gramas")
)

df_produto_enriched = df_produto_agg.join(
    df_produtos_prep,
    df_produto_agg["fk_produto"] == df_produtos_prep["id_produto"],
    "left"
)

produtos_sem_categoria = df_produto_enriched.filter(col("categoria_produto").isNull()).count()
print(f"\nProdutos sem categoria apos join: {produtos_sem_categoria:,}")

if produtos_sem_categoria > 0:
    print(f"ALERTA: {produtos_sem_categoria:,} produtos vendidos nao possuem categoria no catalogo")
else:
    print("OK: Todos os produtos possuem categoria")

print("\n[ETAPA 64] Enriquecimento concluido")

### Etapa 5: Calculo de Metricas Derivadas

Calcula o preco medio em BRL (receita_brl / quantidade_vendida) aplicando protecao contra divisao por zero. Esta metrica identifica produtos com pricing premium ou desconto agressivo. Apos o calculo, seleciona e renomeia as colunas conforme a estrutura definida da view (id_produto, categoria_produto, quantidade_vendida, total_pedidos, receita_brl, receita_usd, preco_medio_brl, avaliacao_media, peso_medio_gramas).

In [0]:
print("[ETAPA 3.6.5] Calculo de metricas derivadas")

df_view_top_produto = df_produto_enriched.withColumn(
    "preco_medio_brl",
    spark_round(
        when(col("quantidade_vendida") > 0, col("receita_brl") / col("quantidade_vendida"))
        .otherwise(lit(0)),
        2
    )
).select(
    col("fk_produto").alias("id_produto"),
    col("categoria_produto"),
    col("quantidade_vendida"),
    col("total_pedidos"),
    col("receita_brl"),
    col("receita_usd"),
    col("preco_medio_brl"),
    col("avaliacao_media"),
    spark_round(col("peso_produto_gramas"), 2).alias("peso_medio_gramas")
)

total_produtos_view = df_view_top_produto.count()
print(f"\nTotal de produtos na view: {total_produtos_view:,}")

print("\nSchema da view:")
df_view_top_produto.printSchema()

print("\n[ETAPA 65] Metricas calculadas")

### Etapa 6: Validacoes de Qualidade

Valida a qualidade dos dados calculados verificando: distribuicao de valores nulos por coluna (categoria e peso podem ter nulos para produtos orfaos), consistencia das metricas calculadas (preco_medio_brl = receita_brl / quantidade_vendida), ranges de valores (avaliacoes entre 0-5, precos positivos), e identificacao de outliers. Esta validacao garante que a view forneca metricas confiaveis para tomada de decisao.

In [0]:
print("[ETAPA 66] Validacoes de qualidade dos dados")

print("\nDistribuicao de valores nulos:")
null_data = []
for col_name in df_view_top_produto.columns:
    null_count = df_view_top_produto.filter(
        col(col_name).isNull()
    ).count()
    null_pct = (null_count / total_produtos_view) * 100
    null_data.append((col_name, null_count, null_pct))

df_nulos = spark.createDataFrame(
    null_data,
    ["coluna", "valores_nulos", "percentual"]
)
display(df_nulos)

print("\nValidacoes de integridade:")
avaliacoes_invalidas = df_view_top_produto.filter(
    (col("avaliacao_media").isNotNull()) &
    ((col("avaliacao_media") < 0) | (col("avaliacao_media") > 5))
).count()

precos_invalidos = df_view_top_produto.filter(
    (col("preco_medio_brl").isNotNull()) &
    (col("preco_medio_brl") < 0)
).count()

receitas_inconsistentes = df_view_top_produto.filter(
    (col("receita_brl") > 0) & (col("quantidade_vendida") == 0)
).count()

df_validacoes = spark.createDataFrame(
    [
        ("Avaliacoes fora do range [0-5]", avaliacoes_invalidas),
        ("Precos medios negativos", precos_invalidos),
        ("Produtos com receita mas quantidade zero", receitas_inconsistentes)
    ],
    ["validacao", "total_inconsistencias"]
)
display(df_validacoes)

df_estatisticas = df_view_top_produto.select(
    "quantidade_vendida",
    "receita_brl",
    "preco_medio_brl",
    "avaliacao_media"
).summary("count", "mean", "stddev", "min", "max")
display(df_estatisticas)

print("\n[ETAPA 66] Validacoes concluidas")
print("\nEstatisticas descritivas:")

### Etapa 7: Criacao da View

Cria a view Delta gold.view_top_produto gravando os dados validados no formato Delta para garantir ACID properties e versionamento. A view consolida as metricas de desempenho por produto permitindo analises de top performers, comparacoes de categorias, e identificacao de produtos com baixa avaliacao ou problemas de pricing. Utilizada pela equipe de gestao de produtos para decisoes estrategicas.

In [0]:
print("[ETAPA 3.6.7] Criacao da view Delta")

view_name = f"{catalog_name}.{gold_db_name}.view_top_produto"

df_view_top_produto.write.format("delta").mode("overwrite").saveAsTable(view_name)

print(f"\nView criada: {view_name}")
print(f"Formato: Delta")
print(f"Modo: Overwrite")

registros_gravados = spark.table(view_name).count()
print(f"Registros gravados: {registros_gravados:,}")

print("\n[ETAPA 67] View criada com sucesso")

### Etapa 8: Validacao Final

Valida a view criada exibindo os top 10 produtos por receita BRL, distribuicao de produtos por categoria, e metricas agregadas gerais. Esta validacao confirma que a view esta acessivel, contem os dados esperados, e fornece insights imediatos sobre os produtos de maior desempenho no catalogo. Permite verificar se as agregacoes e joins foram executados corretamente.

In [0]:
print("[ETAPA 3.6.8] Validacao final da view")

df_view_final = spark.table(view_name)

print("\nTop 10 produtos por receita BRL:")
df_top_10 = df_view_final.orderBy(col("receita_brl").desc()).limit(10)
display(df_top_10)

print("\nDistribuicao de produtos por categoria:")
df_categoria = df_view_final.groupBy("categoria_produto").agg(
    F.count("id_produto").alias("total_produtos"),
    spark_round(F.sum("receita_brl"), 2).alias("receita_total_brl")
).orderBy(col("receita_total_brl").desc())
display(df_categoria)

print("\nSumario de metricas gerais:")
df_sumario = df_view_final.agg(
    F.count("id_produto").alias("total_produtos_unicos"),
    F.sum("quantidade_vendida").alias("quantidade_total_vendida"),
    F.sum("total_pedidos").alias("total_pedidos"),
    spark_round(F.sum("receita_brl"), 2).alias("receita_total_brl"),
    spark_round(F.sum("receita_usd"), 2).alias("receita_total_usd"),
    spark_round(F.avg("preco_medio_brl"), 2).alias("preco_medio_geral_brl"),
    spark_round(F.avg("avaliacao_media"), 2).alias("avaliacao_media_geral")
)
display(df_sumario)

print("\n[ETAPA 68] Validacao concluida")
print("="*80)

## Questao 3.5 - view_vendas_produtos_esteticos

**Descricao:**
View analitica para acompanhamento mensal de vendas de produtos da categoria Fashion. Consolida metricas de receita, volume de pedidos e avaliacao media por mes e categoria, permitindo a area de Fashion identificar tendencias sazonais, desempenho de categorias especificas e oportunidades de crescimento.

**Origem dos Dados:**
- `gold.ft_vendas_geral`: Base de vendas com itens, precos BRL/USD, avaliacoes e datas
- `gold.dm_tempo`: Dimensao temporal para agregacao por ano/mes
- `silver.ft_produtos`: Catalogo de produtos para filtro de categorias fashion

**Estrutura da View:**
| Campo | Tipo | Descricao |
|-------|------|-----------|
| ano | INT | Ano da venda (dm_tempo) |
| mes | INT | Mes numerico 1-12 (dm_tempo) |
| categoria_produto | STRING | Categoria iniciando com "fashion_" |
| total_pedidos | BIGINT | Pedidos distintos no periodo |
| total_itens_vendidos | BIGINT | Quantidade total de itens vendidos |
| receita_total_brl | DECIMAL(12,2) | Soma receita em BRL |
| receita_total_usd | DECIMAL(12,2) | Soma receita em USD (cotacao pedido) |
| ticket_medio_brl | DECIMAL(12,2) | Receita BRL / itens vendidos |
| ticket_medio_usd | DECIMAL(12,2) | Receita USD / itens vendidos |
| avaliacao_media | DECIMAL(3,2) | Media avaliacoes pedidos entregues |

**Regras de Negocio:**
- Filtro obrigatorio: categoria_produto LIKE 'fashion%'
- Agregacao: ano + mes + categoria_produto
- Ticket medio: protecao contra divisao por zero
- Avaliacao media: considerar apenas avaliacoes validas (NOT NULL)
- Join temporal: via fk_tempo → sk_tempo

**Proposito:**
Fornece visao consolidada de performance mensal para equipe Fashion tomar decisoes sobre mix de produtos, acoes promocionais e investimentos em categorias especificas. Permite comparacao de receita BRL vs USD para avaliar impacto cambial nas vendas.

### Estrutura de CTEs

A implementacao utiliza Common Table Expressions (CTEs) para modularizar a logica de transformacao e agregacao, seguindo o padrao SQL recomendado para views complexas. As CTEs sao executadas em ordem sequencial:

**CTE 1 - produtos_fashion:**
Filtra produtos do catalogo cuja categoria inicia com 'fashion', obtendo id_produto e categoria_produto. Esta CTE reduz o volume de dados logo no inicio da pipeline, melhorando performance das operacoes subsequentes. Esta filtragem garante que apenas produtos da linha fashion sejam considerados, excluindo outras categorias como eletronicos, casa, etc. A categoria 'fashion' inclui subcategorias como fashion_bolsas, fashion_calcados, fashion_roupa, etc., representando o segmento de moda e acessorios da empresa.

**CTE 2 - vendas_fashion:**
Realiza join entre ft_vendas_geral e produtos_fashion, selecionando apenas vendas de produtos fashion. Mantem campos necessarios: fk_tempo, id_pedido, id_item, categoria, valores BRL/USD e avaliacao. Esta CTE consolida a base de vendas filtrada. O join inner garante que apenas vendas de produtos fashion sejam incluidas, filtrando transacoes de outras categorias. Os campos selecionados incluem identificadores unicos (id_pedido, id_item), chave temporal (fk_tempo), categoria para agrupamento, valores monetarios em ambas moedas para analises cambiais, e avaliacao para medir satisfacao do cliente com produtos fashion.

**CTE 3 - vendas_tempo:**
Enriquece vendas_fashion com atributos temporais (ano, mes) atraves de join com dm_tempo via fk_tempo → sk_tempo. Esta CTE adiciona a dimensao temporal necessaria para agregacao mensal. O join com dm_tempo adiciona contexto temporal padronizado, permitindo agregacoes consistentes por periodos. Os atributos ano e mes sao essenciais para analises sazonais, identificacao de tendencias mensais e comparacoes interanuais no segmento fashion.

**CTE 4 - agregacao_mensal:**
Agrega vendas por ano, mes e categoria calculando: total_pedidos (countDistinct), total_itens_vendidos (count), receitas BRL/USD (sum), avaliacao_media (avg). Esta CTE gera as metricas brutas antes dos calculos derivados. A agregacao por ano, mes e categoria permite analises multidimensionais: performance mensal por categoria, comparacao interanual, identificacao de categorias em crescimento ou declinio. As metricas calculadas fornecem indicadores chave de negocio: volume de vendas, receita em multiplas moedas e satisfacao media dos clientes.

**CTE 5 - metricas_finais:**
Calcula ticket medio BRL e USD (receita_total / total_itens) aplicando protecao contra divisao por zero. Esta CTE produz o dataset final com todas as 10 colunas da view. O ticket medio e um indicador fundamental de valor por transacao, revelando se o segmento fashion esta focado em produtos premium (ticket alto) ou volume (ticket baixo). A protecao contra divisao por zero evita erros em periodos sem vendas. As metricas finais permitem benchmarking entre categorias e periodos, suportando decisoes estrategicas de pricing e mix de produtos.

### Etapa 1: Validacao de Categorias Fashion

Valida a existencia de produtos fashion no catalogo identificando quantas categorias iniciam com 'fashion' e o total de produtos nessas categorias. Esta validacao garante que o filtro da view tera dados suficientes para analise. Exibe amostra das categorias fashion disponiveis.

In [0]:
print("[ETAPA 3.7.1] Validacao de categorias fashion no catalogo")

df_produtos_catalogo = validar_tabela_silver("ft_produtos")

df_fashion_categorias = df_produtos_catalogo.filter(
    col("categoria_produto").startswith("fashion")
).select("categoria_produto").distinct()

total_categorias_fashion = df_fashion_categorias.count()
print(f"\nTotal de categorias fashion: {total_categorias_fashion}")

if total_categorias_fashion == 0:
    print("ALERTA: Nenhuma categoria fashion encontrada no catalogo")
else:
    print(f"OK: {total_categorias_fashion} categorias fashion disponiveis")
    print("\nCategorias fashion identificadas:")
    display(df_fashion_categorias.orderBy("categoria_produto"))

df_produtos_fashion = df_produtos_catalogo.filter(
    col("categoria_produto").startswith("fashion")
)
total_produtos_fashion = df_produtos_fashion.count()
print(f"\nTotal de produtos fashion no catalogo: {total_produtos_fashion:,}")

print("\n[ETAPA 3.7.1] Validacao concluida")

### Etapa 2: Validacao de Vendas Fashion

Valida a existencia de vendas de produtos fashion na tabela fato, cruzando ft_vendas_geral com produtos fashion. Verifica se ha itens vendidos dessas categorias e exibe amostra com valores, datas e avaliacoes. Esta validacao confirma que a view tera dados para todos os periodos analisados.

In [0]:
print("[ETAPA 3.7.2] Validacao de vendas de produtos fashion")

df_vendas = spark.table(f"{catalog_name}.{gold_db_name}.ft_vendas_geral")

df_vendas_fashion_sample = df_vendas.join(
    df_produtos_fashion.select("id_produto", "categoria_produto"),
    df_vendas["fk_produto"] == df_produtos_fashion["id_produto"],
    "inner"
)

total_itens_fashion = df_vendas_fashion_sample.count()
print(f"\nTotal de itens vendidos (categoria fashion): {total_itens_fashion:,}")

if total_itens_fashion == 0:
    print("ALERTA: Nenhum item fashion vendido encontrado")
else:
    print(f"OK: {total_itens_fashion:,} itens fashion vendidos disponiveis para analise")
    
    pedidos_fashion = df_vendas_fashion_sample.select("id_pedido").distinct().count()
    print(f"Total de pedidos com produtos fashion: {pedidos_fashion:,}")
    
    print("\nAmostra de vendas fashion:")
    df_sample = df_vendas_fashion_sample.select(
        "fk_tempo", "id_pedido", "categoria_produto", 
        "valor_produto_brl", "valor_produto_usd", "avaliacao_pedido"
    ).limit(5)
    display(df_sample)

print("\n[ETAPA 3.7.2] Validacao concluida")

### Etapa 3: Criacao da View com CTEs

Implementa a view usando SQL com CTEs (Common Table Expressions) para estruturar a logica em etapas modulares. Cada CTE representa uma transformacao especifica: filtragem de produtos fashion, join com vendas, enriquecimento temporal, agregacao mensal e calculo de metricas derivadas. A query final consolida todas as transformacoes em uma view Delta.

In [0]:
print("[ETAPA 3.7.3] Criacao da view com CTEs")

query_view_fashion = f"""
WITH produtos_fashion AS (
    SELECT 
        id_produto,
        categoria_produto
    FROM {catalog_name}.silver.ft_produtos
    WHERE categoria_produto LIKE 'fashion%'
),
vendas_fashion AS (
    SELECT 
        v.fk_tempo,
        v.id_pedido,
        v.id_item,
        p.categoria_produto,
        v.valor_produto_brl,
        v.valor_produto_usd,
        v.avaliacao_pedido
    FROM {catalog_name}.{gold_db_name}.ft_vendas_geral v
    INNER JOIN produtos_fashion p ON v.fk_produto = p.id_produto
),
vendas_tempo AS (
    SELECT 
        t.ano,
        t.mes,
        v.id_pedido,
        v.id_item,
        v.categoria_produto,
        v.valor_produto_brl,
        v.valor_produto_usd,
        v.avaliacao_pedido
    FROM vendas_fashion v
    INNER JOIN {catalog_name}.{gold_db_name}.dm_tempo t ON v.fk_tempo = t.sk_tempo
),
agregacao_mensal AS (
    SELECT 
        ano,
        mes,
        categoria_produto,
        COUNT(DISTINCT id_pedido) AS total_pedidos,
        COUNT(id_item) AS total_itens_vendidos,
        ROUND(SUM(valor_produto_brl), 2) AS receita_total_brl,
        ROUND(SUM(valor_produto_usd), 2) AS receita_total_usd,
        ROUND(AVG(CASE WHEN avaliacao_pedido IS NOT NULL THEN avaliacao_pedido END), 2) AS avaliacao_media
    FROM vendas_tempo
    GROUP BY ano, mes, categoria_produto
),
metricas_finais AS (
    SELECT 
        ano,
        mes,
        categoria_produto,
        total_pedidos,
        total_itens_vendidos,
        receita_total_brl,
        receita_total_usd,
        ROUND(
            CASE 
                WHEN total_itens_vendidos > 0 THEN receita_total_brl / total_itens_vendidos
                ELSE 0 
            END, 
            2
        ) AS ticket_medio_brl,
        ROUND(
            CASE 
                WHEN total_itens_vendidos > 0 THEN receita_total_usd / total_itens_vendidos
                ELSE 0 
            END, 
            2
        ) AS ticket_medio_usd,
        avaliacao_media
    FROM agregacao_mensal
)
SELECT * FROM metricas_finais
ORDER BY ano, mes, categoria_produto
"""

view_fashion_name = f"{catalog_name}.{gold_db_name}.view_vendas_produtos_esteticos"

df_view_fashion = spark.sql(query_view_fashion)
df_view_fashion.write.format("delta").mode("overwrite").saveAsTable(view_fashion_name)

print(f"\nView criada: {view_fashion_name}")
print(f"Formato: Delta")
print(f"Modo: Overwrite")

registros_view_fashion = spark.table(view_fashion_name).count()
print(f"Registros na view: {registros_view_fashion:,}")

print("\n[ETAPA 3.7.3] View criada com sucesso")

### Etapa 4: Validacao do Schema

Valida que a view criada possui exatamente os 10 campos especificados com os tipos de dados corretos: ano (INT), mes (INT), categoria_produto (STRING), total_pedidos (BIGINT), total_itens_vendidos (BIGINT), receita_total_brl (DECIMAL), receita_total_usd (DECIMAL), ticket_medio_brl (DECIMAL), ticket_medio_usd (DECIMAL), avaliacao_media (DECIMAL). Exibe o schema completo para conferencia.

In [0]:
print("[ETAPA 3.7.4] Validacao do schema da view")

df_view_fashion_final = spark.table(view_fashion_name)

print("\nSchema da view:")
df_view_fashion_final.printSchema()

campos_esperados = [
    ("ano", "int"),
    ("mes", "int"),
    ("categoria_produto", "string"),
    ("total_pedidos", "bigint"),
    ("total_itens_vendidos", "bigint"),
    ("receita_total_brl", "decimal"),
    ("receita_total_usd", "decimal"),
    ("ticket_medio_brl", "decimal"),
    ("ticket_medio_usd", "decimal"),
    ("avaliacao_media", "decimal")
]

schema_ok = True
for campo_nome, campo_tipo in campos_esperados:
    campo_encontrado = False
    for field in df_view_fashion_final.schema.fields:
        if field.name == campo_nome:
            campo_encontrado = True
            tipo_atual = str(field.dataType).lower()
            if campo_tipo not in tipo_atual:
                print(f"ALERTA: Campo {campo_nome} com tipo incorreto: {field.dataType}")
                schema_ok = False
            break
    if not campo_encontrado:
        print(f"ERRO: Campo {campo_nome} nao encontrado")
        schema_ok = False

if schema_ok:
    print("\nOK: Schema da view conforme especificado")
else:
    print("\nALERTA: Schema possui diferencas em relacao a especificacao")

print("\n[ETAPA 3.7.4] Validacao de schema concluida")

### Etapa 5: Validacao de Cobertura Temporal

Valida a cobertura temporal da view verificando periodo minimo e maximo (ano/mes), total de meses cobertos e distribuicao de registros por ano. Esta validacao confirma que a view possui dados historicos completos para analise de tendencias e sazonalidade das vendas fashion.

In [0]:
print("[ETAPA 3.7.5] Validacao de cobertura temporal")

print("\nPeriodo coberto pela view:")
df_periodo = df_view_fashion_final.agg(
    F.min("ano").alias("ano_min"),
    F.max("ano").alias("ano_max"),
    F.min("mes").alias("mes_min"),
    F.max("mes").alias("mes_max")
)
display(df_periodo)

print("\nDistribuicao de registros por ano:")
df_dist_ano = df_view_fashion_final.groupBy("ano").agg(
    F.count("*").alias("total_registros"),
    F.countDistinct("categoria_produto").alias("categorias_distintas"),
    spark_round(F.sum("receita_total_brl"), 2).alias("receita_ano_brl")
).orderBy("ano")
display(df_dist_ano)

print("\nTotal de combinacoes ano/mes distintas:")
combinacoes_tempo = df_view_fashion_final.select("ano", "mes").distinct().count()
print(f"Combinacoes ano/mes: {combinacoes_tempo}")

print("\n[ETAPA 3.7.5] Validacao de cobertura temporal concluida")

### Etapa 6: Validacao de Metricas

Valida a integridade das metricas calculadas verificando: valores nulos em campos obrigatorios, consistencia dos tickets medios (receita/itens), avaliacoes dentro do range valido (0-5), e identificacao de outliers. Exibe estatisticas descritivas das metricas principais para analise de distribuicao.

In [0]:
print("[ETAPA 3.7.6] Validacao de integridade das metricas")

print("\nValidacoes de valores nulos em campos obrigatorios:")
campos_obrigatorios = ["ano", "mes", "categoria_produto", "total_pedidos", "total_itens_vendidos"]
null_data = []
for campo in campos_obrigatorios:
    null_count = df_view_fashion_final.filter(col(campo).isNull()).count()
    null_data.append((campo, null_count))

df_null_check = spark.createDataFrame(null_data, ["campo_obrigatorio", "valores_nulos"])
display(df_null_check)

print("\nValidacoes de consistencia:")
receitas_negativas_brl = df_view_fashion_final.filter(col("receita_total_brl") < 0).count()
receitas_negativas_usd = df_view_fashion_final.filter(col("receita_total_usd") < 0).count()
tickets_negativos_brl = df_view_fashion_final.filter(col("ticket_medio_brl") < 0).count()
tickets_negativos_usd = df_view_fashion_final.filter(col("ticket_medio_usd") < 0).count()
avaliacoes_invalidas = df_view_fashion_final.filter(
    (col("avaliacao_media").isNotNull()) &
    ((col("avaliacao_media") < 0) | (col("avaliacao_media") > 5))
).count()

df_validacoes = spark.createDataFrame([
    ("Receitas BRL negativas", receitas_negativas_brl),
    ("Receitas USD negativas", receitas_negativas_usd),
    ("Tickets medios BRL negativos", tickets_negativos_brl),
    ("Tickets medios USD negativos", tickets_negativos_usd),
    ("Avaliacoes fora do range [0-5]", avaliacoes_invalidas)
], ["validacao", "total_inconsistencias"])
display(df_validacoes)

print("\nEstatisticas descritivas das metricas:")
df_estatisticas = df_view_fashion_final.select(
    "total_pedidos", "total_itens_vendidos", "receita_total_brl", 
    "ticket_medio_brl", "avaliacao_media"
).summary("count", "mean", "stddev", "min", "max")
display(df_estatisticas)

print("\n[ETAPA 3.7.6] Validacao de metricas concluida")

### Etapa 7: Analise de Performance por Categoria

Exibe o ranking de categorias fashion por receita total BRL, identificando as categorias de maior performance. Calcula tambem metricas agregadas por categoria para comparacao: receita media mensal, ticket medio e avaliacao media. Esta analise fornece insights sobre quais subcategorias fashion tem melhor desempenho.

In [0]:
print("[ETAPA 3.7.7] Analise de performance por categoria fashion")

print("\nRanking de categorias por receita total BRL:")
df_ranking_categorias = df_view_fashion_final.groupBy("categoria_produto").agg(
    F.sum("total_pedidos").alias("total_pedidos"),
    F.sum("total_itens_vendidos").alias("total_itens_vendidos"),
    spark_round(F.sum("receita_total_brl"), 2).alias("receita_total_brl"),
    spark_round(F.sum("receita_total_usd"), 2).alias("receita_total_usd"),
    spark_round(F.avg("ticket_medio_brl"), 2).alias("ticket_medio_brl"),
    spark_round(F.avg("avaliacao_media"), 2).alias("avaliacao_media_geral")
).orderBy(col("receita_total_brl").desc())
display(df_ranking_categorias)

print("\nTotal de categorias fashion distintas:")
total_categorias = df_view_fashion_final.select("categoria_produto").distinct().count()
print(f"{total_categorias} categorias fashion na view")

print("\n[ETAPA 3.7.7] Analise de performance concluida")

### Etapa 8: Validacao Final da View

Exibe amostra completa da view ordenada por ano, mes e receita para conferencia visual dos dados. Calcula sumario geral consolidando todas as metricas: total de registros, soma de pedidos, itens vendidos, receitas BRL/USD, tickets medios e avaliacao media geral. Esta validacao final confirma que a view esta pronta para uso pela equipe Fashion.

In [0]:
print("[ETAPA 3.7.8] Validacao final da view")

print("\nAmostra da view (top 20 registros por receita):")
df_amostra_final = df_view_fashion_final.orderBy(
    col("ano"), col("mes"), col("receita_total_brl").desc()
).limit(20)
display(df_amostra_final)

print("\nSumario geral de metricas:")
df_sumario_final = df_view_fashion_final.agg(
    F.count("*").alias("total_registros"),
    F.sum("total_pedidos").alias("soma_pedidos"),
    F.sum("total_itens_vendidos").alias("soma_itens_vendidos"),
    spark_round(F.sum("receita_total_brl"), 2).alias("receita_acumulada_brl"),
    spark_round(F.sum("receita_total_usd"), 2).alias("receita_acumulada_usd"),
    spark_round(F.avg("ticket_medio_brl"), 2).alias("ticket_medio_geral_brl"),
    spark_round(F.avg("ticket_medio_usd"), 2).alias("ticket_medio_geral_usd"),
    spark_round(F.avg("avaliacao_media"), 2).alias("avaliacao_media_geral")
)
display(df_sumario_final)

print("\n[ETAPA 3.7.8] Validacao final concluida")
print("="*80)